# Marine Heatwaves in a warming Mediterranean Sea

A temporale analysis

Simon Oiry [](https://orcid.org/0000-0001-7161-5246) (Institut des Substances et Organismes de la Mer, ISOMer, Nantes Université, UR 2160, F-44000 Nantes, France)  
Maria Laura Zoffoli [](https://orcid.org/0000-0003-1892-0051) (Consiglio Nazionale delle Ricerche, Istituto di Scienze Marine (CNR-ISMAR), 00133 Rome, Italy)  
Christian Marchese [](https://orcid.org/0000-0002-2414-9251) (Consiglio Nazionale delle Ricerche, Istituto di Scienze Marine (CNR-ISMAR), 00133 Rome, Italy)  
Laurent Barillé [](https://orcid.org/0000-0001-5138-2684) (Institut des Substances et Organismes de la Mer, ISOMer, Nantes Université, UR 2160, F-44000 Nantes, France)  
July 23, 2025

TBD

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attachement du package : 'tidyterra'

L'objet suivant est masqué depuis 'package:stats':

    filter

terra 1.8.50

Attachement du package : 'terra'

L'objet suivant est masqué depuis 'package:tidyr':

    extract


Attachement du package : 'patchwork'

L'objet suivant est masqué depuis 'package:terra':

    area


Attachement du package : 'gridExtra'

L'objet suivant est masqué depuis 'package:dplyr':

    combine


Attachement du package : 'ggrastr'

L'objet suivant est masqué depuis 'package:terra':

    rasterize

Le chargement a nécessité le package : cowplot

Attachement du package : 'cowplot'

L'objet suivant est masqué depuis 'package:patchwork':

    align_plots

L'objet suivant est masqué depuis 'package:lubridate':

    stamp

Le chargement a nécessité le package : magrittr

Attachement du package : 'magrittr'

Les objets suivants sont masqués depuis 'package:terra':

    extract, inset

L'objet suivant est masqué depuis 'package:purrr':

    set_names

L'objet suivant est masqué depuis 'package:tidyr':

    extract

# 1. Introduction

Marine heatwaves (MHWs) are discrete and prolonged periods of anomalously high sea temperatures that significantly exceed historical baseline conditions. Specifically, a marine heatwave is generally defined as an event lasting five or more consecutive days during which sea temperatures exceed the 90th percentile threshold based on a 30-year historical climatological period (<a href="#fig-HW_explain" class="quarto-xref">Figure 1</a>). These events can vary in terms of duration, intensity, and spatial extent, making a flexible yet rigorous approach necessary for accurate characterization and comparison.

In [ ]:
knitr::include_graphics("Figs/plot_Tmax_flame.png")

The importance of studying marine heatwaves arises from their profound ecological and socioeconomic impacts. They have been associated with widespread disruptions to marine ecosystems, including shifts in species distribution, local extinctions, significant mortality events, and changes in primary productivity. Such ecological disturbances subsequently affect fisheries, aquaculture, and biodiversity, highlighting the broad-reaching implications of these events.

The Mediterranean Sea, characterized as a climate change hotspot, has witnessed an increased frequency and intensity of marine heatwaves, especially over recent decades. Mediterranean MHWs often coincide with atmospheric heatwaves, further exacerbating their intensity and ecological impacts. Concurrent atmospheric and marine heatwaves amplify sea surface temperature anomalies, leading to intense ocean stratification and extensive ecological disruption, including mass mortalities of benthic invertebrates and seagrass decline.

In the Mediterranean, particularly in the Adriatic Sea, studies have documented variable responses of primary productivity to MHW events. In coastal and eutrophic regions, such as areas influenced by riverine nutrient input, MHWs have resulted in increased phytoplankton biomass. Conversely, offshore and oligotrophic regions typically exhibit reduced chlorophyll-a concentrations during heatwaves, indicating a potential decline in phytoplankton productivity. Such spatial heterogeneity emphasizes the complexity of ecological responses and underscores the importance of localized assessments.

Understanding marine heatwaves, particularly in sensitive regions like the Mediterranean Sea, is crucial for developing informed management strategies aimed at mitigating their ecological and socioeconomic consequences. This necessity is heightened by projections indicating further increases in MHW frequency, duration, and intensity, driven by ongoing climate change.

# 2. Material & Methods

Daily Sea Surface Temperature (SST) time series for the Mediterranean Sea, spanning from 1982 onwards, were obtained from the Copernicus Marine Environment Monitoring Service (CMEMS) platform ([Copernicus, 2024](#ref-Copernicus_Sentinel); [Embury et al., 2024](#ref-embury2024satellite); [Pisano et al., 2016](#ref-pisano2016new)). Heatwave events were detected using the HeatwaveR package in R ([Schlegel and Smit, 2018](#ref-heatwaveR)), adhering to the marine heatwave definition provided by Hobday et al. ([2016](#ref-hobday2016hierarchical)) and Hobday et al. ([2018](#ref-hobday2018categorizing)).

To establish a reference climatology, the first 30 years of the dataset were utilized for each pixel within the Mediterranean Sea. Daily SST values were subsequently compared against this baseline climatology. A heatwave event was identified whenever the daily SST exceeded the 90th percentile of the corresponding climatological distribution. Additionally, consecutive events separated by fewer than two days were merged and treated as a single continuous heatwave event.

Results are presented as annual maps of key heatwave metrics: the total number of days classified as heatwave, the number of distinct heatwave events, and the cumulative intensity of these events, defined as the sum of daily SST exceedances above the climatological 90th percentile threshold.

In [ ]:
library(sf)
library(ggplot2)
library(rnaturalearth)
library(rnaturalearthdata)
library(tidyverse)
library(ggspatial)

# Load your shapefile (replace with your path)
med_regions <- st_read("Data/mask_Med_Sea/More_regions_Med_Sea.shp") %>% 
  dplyr::filter(!str_detect(name, "Black"))

# Get world land polygons
world <- ne_countries(scale = "large", returnclass = "sf")

# Plot
plot_medsea <- ggplot() +
  # Plot land as grey
  geom_sf(data = world, fill = "grey80", color = NA) +
  # Plot Mediterranean regions, each with a different color
  geom_sf(data = med_regions, aes(fill = name), color = "black", alpha = 0.8) +
  # Customize colors
  scale_fill_manual(name = "",values = region_col)  +
    ## ── horizontal guide with title on top ──────────────────────
  guides(
    fill = guide_legend(
      ncol = 1,
      byrow      = TRUE,
      title.position = "top",
      label.position = "right",            # optional – labels below bar
      barwidth       = unit(0.7, "cm"),       # tweak length
      barheight      = unit(0.7, "cm")# tweak thickness
    )
  ) +
  ## ── map extent ──────────────────────────────────────────────
  coord_sf(xlim = c(-10, 40), ylim = c(30, 47), expand = FALSE) +
  ## ── *** scale bar *** ───────────────────────────────────────
  annotation_scale(
    location   = "bl",   # bottom-left
    width_hint = 0.25,   # ≈25 % of the map width
    line_width = 0.6,
    text_cex   = 2,
    pad_x      = unit(0.4, "cm"),
    pad_y      = unit(0.4, "cm")
  ) +
  ## ── theme ───────────────────────────────────────────────────
  theme_minimal(base_size = 18) +
  theme(
    panel.grid        = element_line(color = "gray90"),
    panel.background  = element_rect(fill = "aliceblue"),
    plot.background   = element_rect(fill = "white", color = NA),

    # move legend INSIDE, upper-left; anchor by its top-left corner
    legend.position       = c(0.02, 0.98),   # npc coords (x = 2 %, y = 98 %)
    legend.justification  = c(0, 1),
    # legend.direction      = "horizontal",
    legend.direction      = "vertical",      # keeps items vertical
    legend.title.align    = 0.5,             # centre title
    legend.text           = element_text(size = 13),
    legend.title = element_blank(),

    # give legend a white background & a bit of padding
    legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA,linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4)
  )

plot_medsea

ggsave("Manuscript/Figs/Map_MedSea.png", plot_medsea, dpi = 300, height =645*3, width = 1319*3, units = "px")

In [ ]:
knitr::include_graphics("Figs/Map_MedSea.png")

HW metrics where summerised for each regions of the Mediterranean sea, following the nomenclature defined by the Marine Strategy Framework Directive (MSFD, <a href="#fig-MapMedSea" class="quarto-xref">Figure 2</a>).

# 3. Results

## 3.1 SST phenology

In [ ]:
library(data.table)
library(dtplyr)
library(tidyverse)
library(dplyr)
library(duckdb)

List_HW_csv <- list.files("Outputs/SST/Cropped_CSV/", pattern = ".csv",recursive = T, full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(filename = gsub(".*/","",path),
         Date = as.Date(substr(filename,1,8), format = "%Y%m%d"), 
         doy = yday(Date))


for (i in unique(List_HW_csv$doy)) {
  
  print(i)
  
  List <- List_HW_csv %>% 
    dplyr::filter(doy == i)
  
  
  # Create a progress bar
  pb <- txtProgressBar(min = 0, max = nrow(List), style = 3)
  
  # Efficiently read all CSV files and combine them into a single data.table with progress bar
  all_data <- rbindlist(lapply(seq_along(List$path), function(i) {
    # Update the progress bar
    setTxtProgressBar(pb, i)
    
    # Read the CSV file
    fread(List$path[i])
  }), use.names = TRUE, fill = TRUE)
  
  # Close the progress bar
  close(pb)
  
  
  split1 <- all_data %>% 
    mutate(year = year(date)) %>% 
    dplyr::filter(year <= 1991) %>% 
    mutate(split = "1982-1991")
  
  split2 <- all_data %>% 
    mutate(year = year(date)) %>% 
    dplyr::filter(year > 1991 & year <= 2001)%>% 
    mutate(split = "1992-2001")
  
  split3 <- all_data %>% 
    mutate(year = year(date)) %>% 
    dplyr::filter(year > 2001 & year <= 2011)%>% 
    mutate(split = "2002-2011")
  
  split4 <- all_data %>% 
    mutate(year = year(date)) %>% 
    dplyr::filter(year > 2011)%>% 
    mutate(split = "2012-2024")
  
  df_by_split <- all_data %>% 
    mutate(year = year(date),
           split = "all") %>% 
    rbind(split1, split2, split3, split4)
  
  # pheno_date <- df_by_split %>% 
  #   group_by(x,y,split) %>% 
  #   reframe(mean = mean(analysed_sst, na.rm = T),
  #           median = median(analysed_sst, na.rm = T),
  #           sd = sd(analysed_sst, na.rm = T),
  #           p05 = quantile(analysed_sst, probs = 0.05),
  #           p25 = quantile(analysed_sst, probs = 0.25),
  #           p75 = quantile(analysed_sst, probs = 0.75),
  #           p95 = quantile(analysed_sst, probs = 0.95)
  # )
  
  
  # Create a temporary DuckDB connection
con <- dbConnect(duckdb::duckdb(), dbdir = ":memory:")

# Copy the tibble to DuckDB as a lazy table
df_lazy <- copy_to(con, df_by_split, temporary = TRUE)

# Now summarise using lazy evaluation
pheno_date <- df_lazy %>%
  group_by(x, y, split) %>%
  summarise(
    mean   = mean(analysed_sst, na.rm = TRUE),
    median = median(analysed_sst, na.rm = TRUE),
    sd     = sd(analysed_sst, na.rm = TRUE),
    p05    = quantile(analysed_sst, 0.05),
    p25    = quantile(analysed_sst, 0.25),
    p75    = quantile(analysed_sst, 0.75),
    p95    = quantile(analysed_sst, 0.95),
    .groups = "drop"
  ) %>%
  collect() %>% 
  mutate(doy = i)# Pull results back into R
  
rm(con)
rm(df_lazy)

fwrite(pheno_date, paste0("Outputs/SST/SST_Phenology/doy/", "SST_Pheno_DOY_",i,".csv"))
}

In [ ]:
library(sf)
library(rnaturalearth)
library(data.table)
library(viridis)
library(tidyterra)
library(ggpp)



filelist <- list.files("Outputs/SST/SST_Phenology/doy/", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         doy = gsub("[^0-9]","",file) %>% 
           as.numeric()) %>% 
  arrange(doy)


# Initialize min and max
global_min <- Inf
global_max <- -Inf

# Loop through files
for (file in filelist$path) {
  # Read only the column you need (e.g., "SST")
  dt <- fread(file, select = "mean", showProgress = FALSE)
  
  # Update global min and max
  global_min <- min(global_min, min(dt$mean, na.rm = TRUE))
  global_max <- max(global_max, max(dt$mean, na.rm = TRUE))
}

# Output
cat("Global min SST:", global_min, "\n")
cat("Global max SST:", global_max, "\n")

# Load your shapefile (replace with your path)
med_regions <- st_read("Data/mask_Med_Sea/More_regions_Med_Sea.shp") %>% 
  dplyr::filter(!str_detect(name, "Black"))

# Get world land polygons
world <- ne_countries(scale = "large", returnclass = "sf")


#### Plot clock 


# Define DOY
doy <- 1:366
df <- tibble(
  doy = doy,
  angle = -2 * pi * (doy / 366) + pi/2,   # clockwise + 90° rotation
  # angle = -2 * pi * (doy / 366), 
  x = cos(angle),
  y = sin(angle)
)

# Define 15th of each month
dates <- ymd("2024-01-15") + months(0:11)
month_labels <- tibble(
  month = lubridate::month(dates, label = TRUE, locale = "UTC"),
  doy = yday(dates),
  angle = -2 * pi * (doy / 366) + pi/2,
  x = 0.8 * cos(angle),  # inside the circle
  y = 0.8 * sin(angle)
)

# Ticks at 1st of each month
dates_ticks <- ymd("2024-01-01") + months(0:11)
month_ticks <- tibble(
  doy = yday(dates_ticks),
  angle = -2 * pi * (doy / 366) + pi/2,
  x_start = 0.95 * cos(angle),
  y_start = 0.95 * sin(angle),
  x_end = 1.05 * cos(angle),
  y_end = 1.05 * sin(angle)
)


# Plot
raw_clock <- ggplot(df, aes(x = x, y = y)) +
  geom_path(linewidth = 1, color = "steelblue") +
  geom_segment(data = month_ticks, aes(x = x_start, y = y_start, xend = x_end, yend = y_end),
               linewidth = 0.8, color = "black") +
  geom_text(data = month_labels, aes(label = month), size = 4) +
  coord_equal() +
  theme_void()


output_plots <- "Outputs/SST/SST_Phenology/plots/"

for (i in filelist$doy) {
  
  df <- fread(filelist$path[i])
  
  angle_today <- -2 * pi * (i / 366) + pi/2
  point_today <- tibble(
    x = 1.0 * cos(angle_today),
    y = 1.0 * sin(angle_today)
  )  
  
  plot_clock <- raw_clock + 
      geom_point(data = point_today, aes(x = x, y = y), color = "red", size = 4) 
  
  clock_grob <- ggplotGrob(plot_clock)
  
  for (ii in unique(df$split)) {
    
    df_split <- df %>% 
      dplyr::filter(split == ii)
    
    map <- ggplot() +
      geom_tile(data = df_split, mapping = aes(x = x, y = y, fill = mean))+
      geom_sf(data = world, fill = "grey80", color = NA) +
      scale_fill_grass_c("celsius",
                         use_grass_range = FALSE,
                         limits = c(global_min,global_max),
                         name     = paste0("Avg SST (", ii, ")"))+
      ## ── colour scales ───────────────────────────────────────────
      # scale_fill_gradientn(
      #   colours  = turbo(5),
      #   na.value = "transparent",
      #   name     = "Avg number of Event per Year",
      #   limits = c(10,15)
      # ) +
        ## ── horizontal guide with title on top ──────────────────────
      guides(
        fill = guide_colourbar(
          direction      = "horizontal",
          title.position = "top",
          label.position = "bottom",            # optional – labels below bar
          barwidth       = unit(9.5, "cm"),       # tweak length
          barheight      = unit(0.45, "cm")     # tweak thickness
        )
      ) +
      ## ── map extent ──────────────────────────────────────────────
      coord_sf(xlim = c(-10, 40), ylim = c(30, 47), expand = FALSE) +
      ## ── *** scale bar *** ───────────────────────────────────────
      annotation_scale(
        location   = "bl",   # bottom-left
        width_hint = 0.25,   # ≈25 % of the map width
        line_width = 0.6,
        text_cex   = 2,
        pad_x      = unit(0.4, "cm"),
        pad_y      = unit(0.4, "cm")
      ) +
      ## ── theme ───────────────────────────────────────────────────
      theme_minimal(base_size = 18) +
      theme(
        panel.grid        = element_line(color = "gray90"),
        panel.background  = element_rect(fill = "aliceblue"),
        plot.background   = element_rect(fill = "white", color = NA),
    
        # move legend INSIDE, upper-left; anchor by its top-left corner
        legend.position       = c(0.02, 0.98),   # npc coords (x = 2 %, y = 98 %)
        legend.justification  = c(0, 1),
        legend.direction      = "horizontal",
    
        legend.title.align    = 0.5,             # centre title
        legend.text           = element_text(size = 13),
    
        # give legend a white background & a bit of padding
        legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA,linewidth = 0.2),
        legend.margin         = margin(4, 4, 4, 4),
        axis.title = element_blank()
      ) +
  annotation_custom(
    grob  = clock_grob,
    xmin  = -9,   # change as needed for position
    xmax  = -1,   # define width
    ymin  = 36,   # change as needed for vertical placement
    ymax  = 44    # define height
  )
    
    ggsave(paste0(output_plots,"/",ii,"/map_",ii,"_DOY_",i,".png"),map,dpi = 300, height =575*3, width = 1319*3, units = "px")
  }
  
}

In [ ]:
library(tidyverse)
library(magick)

files <- list.files("Outputs/SST/SST_Phenology/plots/1982-1991", pattern = ".png", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         num = gsub(".*_","",file) %>% gsub(".png","",.) %>% as.numeric()) %>% 
  arrange(num) %>% 
  pull(path)

gifski::gifski(files, gif_file = "Outputs/SST/SST_Phenology/plots/gif/GIF_1982-1991.gif",
               delay = 0.1, loop = TRUE, progress = TRUE,width=3957/4,height=1725/4)

files <- list.files("Outputs/SST/SST_Phenology/plots/1992-2001", pattern = ".png", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         num = gsub(".*_","",file) %>% gsub(".png","",.) %>% as.numeric()) %>% 
  arrange(num) %>% 
  pull(path)

gifski::gifski(files, gif_file = "Outputs/SST/SST_Phenology/plots/gif/GIF_1992-2001.gif",
               delay = 0.1, loop = TRUE, progress = TRUE,width=3957/4,height=1725/4)

files <- list.files("Outputs/SST/SST_Phenology/plots/2002-2011", pattern = ".png", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         num = gsub(".*_","",file) %>% gsub(".png","",.) %>% as.numeric()) %>% 
  arrange(num) %>% 
  pull(path)

gifski::gifski(files, gif_file = "Outputs/SST/SST_Phenology/plots/gif/GIF_2002-2011.gif",
               delay = 0.1, loop = TRUE, progress = TRUE,width=3957/4,height=1725/4)

files <- list.files("Outputs/SST/SST_Phenology/plots/2012-2024", pattern = ".png", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         num = gsub(".*_","",file) %>% gsub(".png","",.) %>% as.numeric()) %>% 
  arrange(num) %>% 
  pull(path)

gifski::gifski(files, gif_file = "Outputs/SST/SST_Phenology/plots/gif/GIF_2012-2024.gif",
               delay = 0.1, loop = TRUE, progress = TRUE,width=3957/4,height=1725/4)


#### COmbine 

gif_list1 <- list.files("Outputs/SST/SST_Phenology/plots/1982-1991", full.names = T) %>% 
  as_tibble() %>% 
  rename(split1 = "value") %>% 
  mutate(file = gsub(".*/","",split1),
         num = gsub(".*_","",file) %>% gsub(".png","",.) %>% as.numeric()) %>% 
  arrange(num) %>% 
  dplyr::select(-file)

gif_list2 <- list.files("Outputs/SST/SST_Phenology/plots/2012-2024", full.names = T) %>% 
  as_tibble() %>% 
  rename(split2 = "value") %>% 
  mutate(file = gsub(".*/","",split2),
         num = gsub(".*_","",file) %>% gsub(".png","",.) %>% as.numeric()) %>% 
  arrange(num) %>% 
  dplyr::select(-file)

df_list <- gif_list1 %>% 
  left_join(gif_list2, by = "num")

for (i in df_list$num) {
  
  img1 <- image_read(df_list$split1[i])
  img2 <- image_read(df_list$split2[i])

  combo <- image_append(c(img1, img2),   stack = TRUE)   # one above the other

  image_write(combo, path = paste0("Outputs/SST/SST_Phenology/plots/Merged/combo_",i,".png"))
}

files <- list.files("Outputs/SST/SST_Phenology/plots/Merged", pattern = ".png", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         num = gsub(".*_","",file) %>% gsub(".png","",.) %>% as.numeric()) %>% 
  arrange(num) %>% 
  pull(path)

gifski::gifski(files, gif_file = "Outputs/SST/SST_Phenology/plots/gif/Merged_1982-1991-2012-2024.gif",
               delay = 0.1, loop = TRUE, progress = TRUE,width=3957/2,height=3450/2)

In [ ]:
library(sf)
library(ggplot2)
library(rnaturalearth)
library(rnaturalearthdata)
library(tidyverse)
library(terra)
library(viridisLite)
library(ggspatial)
library(Utilities.Package)
library(tidyterra)

filelist <- list.files("Outputs/SST/SST_Phenology/doy/", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         doy = gsub("[^0-9]","",file) %>% 
           as.numeric()) %>% 
  arrange(doy)


for (i in filelist$doy) {
  
  df <- fread(filelist$path[i]) %>% 
    dplyr::filter(split %in% c("1982-1991","2012-2024")) %>% 
    dplyr::select(x,y,split,mean) %>% 
    pivot_wider(names_from = split, values_from = mean) %>% 
    mutate(diff = `2012-2024` - `1982-1991`, 
           doy = i) %>%
    dplyr::select(x,y,diff,doy)
    

  fwrite(df,paste0("Outputs/SST/SST_Phenology/diff_splits/Diff_1982-1991-2012-2024_DOY_",i,".csv"))
}



filelist <- list.files("Outputs/SST/SST_Phenology/diff_splits/", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         doy = gsub(".*_","",file) %>% gsub(".csv","",.) %>% 
           as.numeric()) %>% 
  arrange(doy)


# Initialize min and max
global_min <- Inf
global_max <- -Inf

# Loop through files
for (file in filelist$path) {
  # Read only the column you need (e.g., "SST")
  dt <- fread(file, select = "diff", showProgress = FALSE)
  
  # Update global min and max
  global_min <- min(global_min, min(dt$diff, na.rm = TRUE))
  global_max <- max(global_max, max(dt$diff, na.rm = TRUE))
}

# Load your shapefile (replace with your path)
med_regions <- st_read("Data/mask_Med_Sea/More_regions_Med_Sea.shp") %>% 
  dplyr::filter(!str_detect(name, "Black"))

# Get world land polygons
world <- ne_countries(scale = "large", returnclass = "sf")


#### Plot clock 


# Define DOY
doy <- 1:366
df <- tibble(
  doy = doy,
  angle = -2 * pi * (doy / 366) + pi/2,   # clockwise + 90° rotation
  # angle = -2 * pi * (doy / 366), 
  x = cos(angle),
  y = sin(angle)
)

# Define 15th of each month
dates <- ymd("2024-01-15") + months(0:11)
month_labels <- tibble(
  month = lubridate::month(dates, label = TRUE, locale = "UTC"),
  doy = yday(dates),
  angle = -2 * pi * (doy / 366) + pi/2,
  x = 0.8 * cos(angle),  # inside the circle
  y = 0.8 * sin(angle)
)

# Ticks at 1st of each month
dates_ticks <- ymd("2024-01-01") + months(0:11)
month_ticks <- tibble(
  doy = yday(dates_ticks),
  angle = -2 * pi * (doy / 366) + pi/2,
  x_start = 0.95 * cos(angle),
  y_start = 0.95 * sin(angle),
  x_end = 1.05 * cos(angle),
  y_end = 1.05 * sin(angle)
)


# Plot
raw_clock <- ggplot(df, aes(x = x, y = y)) +
  geom_path(linewidth = 1, color = "steelblue") +
  geom_segment(data = month_ticks, aes(x = x_start, y = y_start, xend = x_end, yend = y_end),
               linewidth = 0.8, color = "black") +
  geom_text(data = month_labels, aes(label = month), size = 4) +
  coord_equal() +
  theme_void()


output_plots <- "Outputs/SST/SST_Phenology/plots/"

for (i in filelist$doy) {
  
  df <- fread(filelist$path[i])
  
  angle_today <- -2 * pi * (i / 366) + pi/2
  point_today <- tibble(
    x = 1.0 * cos(angle_today),
    y = 1.0 * sin(angle_today)
  )  
  
  plot_clock <- raw_clock + 
      geom_point(data = point_today, aes(x = x, y = y), color = "red", size = 4) 
  
  clock_grob <- ggplotGrob(plot_clock)
  

    
    map <- ggplot() +
      geom_tile(data = df, mapping = aes(x = x, y = y, fill = diff))+
      geom_sf(data = world, fill = "grey80", color = NA) +
      scale_fill_gradientn(
  name = "SST mean difference\n1982–1991 to 2012–2024",
  colours = c("#2166AC", "#67A9CF", "#D1E5F0",  # Negative (dark to light blue)
              "white",                         # Midpoint (0)
              "#4FF76B", "#088548","#FFBA0A","#D40B0B", "#990E99"),# Positive (light to dark red)
  values = scales::rescale(c(global_min, -1, -0.1, 0, 0.1, 1, 2,3, global_max)),
  limits = c(global_min, global_max)
)+

      guides(
        fill = guide_colourbar(
          direction      = "horizontal",
          title.position = "top",
          label.position = "bottom",            # optional – labels below bar
          barwidth       = unit(9.5, "cm"),       # tweak length
          barheight      = unit(0.45, "cm")     # tweak thickness
        )
      ) +
      ## ── map extent ──────────────────────────────────────────────
      coord_sf(xlim = c(-10, 40), ylim = c(30, 47), expand = FALSE) +
      ## ── *** scale bar *** ───────────────────────────────────────
      annotation_scale(
        location   = "bl",   # bottom-left
        width_hint = 0.25,   # ≈25 % of the map width
        line_width = 0.6,
        text_cex   = 2,
        pad_x      = unit(0.4, "cm"),
        pad_y      = unit(0.4, "cm")
      ) +
      ## ── theme ───────────────────────────────────────────────────
      theme_minimal(base_size = 18) +
      theme(
        panel.grid        = element_line(color = "gray90"),
        panel.background  = element_rect(fill = "aliceblue"),
        plot.background   = element_rect(fill = "white", color = NA),
    
        # move legend INSIDE, upper-left; anchor by its top-left corner
        legend.position       = c(0.02, 0.98),   # npc coords (x = 2 %, y = 98 %)
        legend.justification  = c(0, 1),
        legend.direction      = "horizontal",
    
        legend.title.align    = 0.5,             # centre title
        legend.text           = element_text(size = 13),
    
        # give legend a white background & a bit of padding
        legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA,linewidth = 0.2),
        legend.margin         = margin(4, 4, 4, 4),
        axis.title = element_blank()
      ) +
  annotation_custom(
    grob  = clock_grob,
    xmin  = -9,   # change as needed for position
    xmax  = -1,   # define width
    ymin  = 35.5,   # change as needed for vertical placement
    ymax  = 43.5    # define height
  )
    
    ggsave(paste0(output_plots,"/Difference/map_differences_DOY_",i,".png"),map,dpi = 300, height =575*3, width = 1319*3, units = "px")

  
}

  

files <- list.files("Outputs/SST/SST_Phenology/plots/Difference", pattern = ".png", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         num = gsub(".*_","",file) %>% gsub(".png","",.) %>% as.numeric()) %>% 
  arrange(num) %>% 
  pull(path)

gifski::gifski(files, gif_file = "Outputs/SST/SST_Phenology/plots/gif/Difference_1982-1991-2012-2024.gif",
               delay = 0.1, loop = TRUE, progress = TRUE,width=3957/2,height=1725/2)

In [ ]:
library(sf)
library(rnaturalearth)
library(data.table)
# Load your shapefile (replace with your path)
med_regions <- st_read("Data/mask_Med_Sea/More_regions_Med_Sea.shp") %>% 
  dplyr::filter(!str_detect(name, "Black"))

# Get world land polygons
world <- ne_countries(scale = "large", returnclass = "sf")

model_rast <- "Outputs/SST/nc/n_event_per_year/n_event_1982.nc" %>% 
  rast()

values(model_rast) <- NA
names(model_rast)<- "val"

filelist <- list.files("Outputs/SST/SST_Phenology/doy/", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(file = gsub(".*/","",path),
         doy = gsub("[^0-9]","",file) %>% 
           as.numeric()) %>% 
  arrange(doy)


for (i in 215:max(filelist$doy)) {
  
  df_i <- fread(filelist$path[i]) %>% 
    dplyr::filter(split != "all") %>% 
    dplyr::select(-doy)

  for (ii in unique(df_i$split)) {
    
    df_ii <- df_i %>% 
      dplyr::filter(split == ii) %>% 
      dplyr::select(-split)
    
    pts <- vect(df_ii, geom = c("x", "y"), crs = crs(model_rast))

    rast_df <- terra::rasterize(
      pts,             # what to burn
      model_rast,         # target raster geometry
      field = c("mean",   "median" ,"sd"   ,  "p05"   , "p25" ,   "p75"  ,  "p95"), # column to transfer
      fun   = "mean"   # how to handle duplicate points (mean, max, sum, …)
    )

    names(rast_df) <- c("mean","median","sd","p05","p25","p75","p95")
  
    writeRaster(rast_df, paste0("Outputs/SST/SST_Phenology/Zonal_Statistics/rast/",ii,"/Rast_Pheno_",ii,"_DOY_",i,".tif"),overwrite = T)
    }
}

In [ ]:
library(sf)
library(rnaturalearth)
library(data.table)
library(terra)
library(progress)

med_regions <- st_read("Data/mask_Med_Sea/More_regions_Med_Sea.shp") %>% 
  dplyr::filter(!str_detect(name, "Black")) %>% 
  dplyr::select(-Id) %>% 
  vect()

med_regions_sf <- st_read("Data/mask_Med_Sea/More_regions_Med_Sea.shp") %>% 
  dplyr::filter(!str_detect(name, "Black")) %>% 
  dplyr::select(-c(Id)) %>% 
  as_tibble() %>% 
  dplyr::select(- geometry)

if (!"poly_id" %in% names(med_regions)) med_regions$poly_id <- seq_len(nrow(med_regions))
if (!"poly_id" %in% names(med_regions_sf)) med_regions_sf$poly_id <- seq_len(nrow(med_regions_sf))

folderlist <- list.files("Outputs/SST/SST_Phenology/Zonal_Statistics/rast/", full.names = T) %>% 
  as_tibble() %>% 
  rename(path = "value") %>% 
  mutate(split = gsub(".*/","",path))


#––––– OUTER progress bar (splits) –––––#
pb_split <- progress_bar$new(
  total   = length(folderlist$split),
  clear   = FALSE,
  format  = "Split :current/:total [:bar] :percent  elapsed=:elapsed"
)

first <- TRUE   # flag to initialise df_out the first time

for (split in folderlist$split) {
  pb_split$tick()   # advance outer bar
  
  split_path <- file.path(
    "Outputs", "SST", "SST_Phenology", "Zonal_Statistics", "rast", split
  )
  
  filelist_split <- list.files(split_path, full.names = TRUE) %>% 
    as_tibble() %>% 
    rename(path = value) %>% 
    mutate(
      file = basename(path),
      doy  = sub(".*_(\\d+)\\.tif$", "\\1", file) |> as.numeric()
    ) %>% 
    arrange(doy)
  
  #––––– INNER progress bar (doys within this split) –––––#
  pb_doy <- progress_bar$new(
    total  = nrow(filelist_split),
    clear  = FALSE,
    format = sprintf("  -> %s :current/:total [:bar] :percent  elapsed=:elapsed", split)
  )
  
  for (i in seq_len(nrow(filelist_split))) {
    pb_doy$tick()   # advance inner bar
    
    img <- rast(filelist_split$path[i])
    doy <- filelist_split$doy[i]
    
    m <- terra::extract(
           img, med_regions,
           fun = mean, na.rm = TRUE
         )
    
    p_means <- cbind(
                 med_regions_sf,
                 m[match(med_regions_sf$poly_id, m$ID), -1]
               ) %>% 
               mutate(doy = doy,
                      split = split) %>% 
               select(-poly_id)
    
    if (first) {
      df_out <- p_means
      first  <- FALSE
    } else {
      df_out <- bind_rows(df_out, p_means)
    }
  }
  pb_doy$terminate()  # finish inner bar for this split
}
pb_split$terminate() 

fwrite(df_out,"Outputs/SST/SST_Phenology/Zonal_Statistics/zonal_stats_DOY_decades.csv")

In [ ]:
library(ggplot2)
library(dplyr)
library(scico) 

df <- fread("Outputs/SST/SST_Phenology/Zonal_Statistics/zonal_stats_DOY_decades.csv") %>% 
  as_tibble()


df %>% 
  ggplot(aes(x = doy, y = mean, color = split, fill = split)) +
  geom_ribbon(aes(ymin = mean - sd, ymax = mean + sd), 
              alpha = 0.15, color = NA) +
  geom_smooth(method = "gam", se = FALSE, linewidth = 1) +
  geom_point(alpha = 0.1, size = 0.5) +
  facet_wrap(~name, scales = "free_y") +
  scale_color_scico_d(palette = "batlow") +
  scale_fill_scico_d(palette = "batlow") +
  labs(
    x = "Day of Year",
    y = "Mean SST (°C)",
    color = "Period",
    fill = "Period",
    title = "Seasonal SST Trends with Variability Bands"
  ) +
  theme_minimal(base_size = 14) +
  theme(
    panel.grid.minor = element_blank(),
    strip.text = element_text(face = "bold", size = 12),
    legend.position = "top",
    legend.title = element_text(size = 12)
  )

df %>%
  ggplot(aes(x = doy, y = mean, color = split)) +
  geom_smooth(method = "gam", se = FALSE, linewidth = 1.2) +
  facet_wrap(~name) +
  scale_color_scico_d(palette = "batlow") +
  labs(
    x = "Day of Year",
    y = "Mean SST (°C)",
    color = "Period",
    title = "Smoothed Seasonal SST Trends by Region"
  ) +
  theme_minimal(base_size = 14) +
  theme(
    strip.text = element_text(face = "bold"),
    legend.position = "top"
  )

In [ ]:
library(terra)
library(dplyr)
library(purrr)
library(progress)

# Load regions
med_regions <- sf::read_sf("Data/mask_Med_Sea/More_regions_Med_Sea.shp")
med_regions$region_ID <- 1:nrow(med_regions)

# List all .nc files and extract date
nc_files <- list.files("Data/SST", pattern = "\\.nc$", full.names = TRUE, recursive = TRUE) %>% 
  tibble(path = .) %>%
  mutate(file = basename(path),
         date = substr(file, 1, 8))  # adapt if date format is different

# Initialize list to store results
results_list <- vector("list", length = nrow(nc_files))

# Create progress bar
pb <- progress_bar$new(
  total = nrow(nc_files),
  format = "[:bar] :percent | :current/:total files | ETA: :eta",
  clear = FALSE,
  width = 60
)

# Loop efficiently
for (i in seq_len(nrow(nc_files))) {
  pb$tick()

  # Load raster
  sst <- rast(nc_files$path[i], subds = 1)

  # Extract SST values by region
  extracted <- terra::extract(sst, med_regions)

  # Calculate statistics
  stats <- extracted %>%
    rename(value = 2, region_ID = ID) %>%
    group_by(region_ID) %>%
    summarise(
      mean   = mean(value, na.rm = TRUE),
      median = median(value, na.rm = TRUE),
      sd     = sd(value, na.rm = TRUE),
      p10    = quantile(value, 0.10, na.rm = TRUE),
      p25    = quantile(value, 0.25, na.rm = TRUE),
      p75    = quantile(value, 0.75, na.rm = TRUE),
      p90    = quantile(value, 0.90, na.rm = TRUE),
      .groups = "drop"
    ) %>%
    mutate(date = nc_files$date[i])

  results_list[[i]] <- stats

  rm(sst, extracted, stats)
  gc()  # clean memory
}

# Combine all results
df_out <- bind_rows(results_list)

# Optional: Add region names
region_info <- as.data.frame(med_regions)[, c("region_ID", "name")]
final_df <- left_join(df_out, region_info, by = "region_ID")

# ---- Save results ----
write.csv(final_df, "Outputs/SST/region_stats.csv", row.names = F)

In [ ]:
library(lubridate)
library(zoo)
library(trend) 
library(tidyverse)
library(data.table)

df_simon <- fread("Outputs/SST/region_stats.csv") %>% 
  mutate(date = as.character(date) %>% as.Date(format = "%Y%m%d"),
         mean = mean - 273.15,
         median = median - 273.15,
         p10 = p10 - 273.15,
         p25 = p25 - 273.15,
         p75 = p75 - 273.15,
         p90 = p90 - 273.15) %>%
  as_tibble() %>% 
  rename(region = "name") %>% 
  pivot_longer(-c(region_ID,date,region), names_to = "metric", values_to = "value") %>% 
  mutate(region_ID = as.character(region_ID)) %>% 
  rename(value_simon = value)


regionID <- df_simon %>% 
  group_by(region_ID) %>% 
  reframe(region = unique(region))

path_file_chouchou <- rstudioapi::selectFile()

df_chouchou <-fread(path_file_chouchou) %>% 
  as_tibble()%>%
  pivot_longer(
    cols       = -Date,
    names_to   = c("metric", "region_ID"),
    names_pattern = "(.*)_region_(\\d+)",
    values_to  = "value"
  ) %>% 
  rename(date = "Date") %>% 
  left_join(regionID, by = "region_ID") %>% 
  mutate(date = as.character(date) %>% as.Date(format = "%Y%m%d"),
         metric = tolower(metric),
         metric = case_when(metric == "std" ~ "sd",
                            T ~ metric)) %>% 
  rename(value_chouchou = value)

df_both <- df_simon %>% 
  left_join(df_chouchou, by = c("date","metric","region_ID","region")) %>% 
  mutate(difference = (value_chouchou - value_simon) / value_simon * 100)


compare <- df_both %>% 
  group_by(region,metric) %>% 
  reframe(avg = mean(difference))


fwrite(df_both,paste0(rstudioapi::selectDirectory(),"/compare_extraction.csv"))

In [ ]:
library(lubridate)
library(zoo)


Attachement du package : 'zoo'

L'objet suivant est masqué depuis 'package:terra':

    time<-

Les objets suivants sont masqués depuis 'package:base':

    as.Date, as.Date.numeric


Attachement du package : 'data.table'

Les objets suivants sont masqués depuis 'package:zoo':

    yearmon, yearqtr

L'objet suivant est masqué depuis 'package:terra':

    shift

Les objets suivants sont masqués depuis 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

Les objets suivants sont masqués depuis 'package:dplyr':

    between, first, last

L'objet suivant est masqué depuis 'package:purrr':

    transpose

In [ ]:
df_timeserie <- fread("../Outputs/SST/region_stats.csv") %>% 
  mutate(date = as.character(date) %>% as.Date(format = "%Y%m%d"),
         mean = mean - 273.15,
         median = median - 273.15,
         p10 = p10 - 273.15,
         p25 = p25 - 273.15,
         p75 = p75 - 273.15,
         p90 = p90 - 273.15) %>%
  as_tibble() %>% 
  mutate(doy = yday(date),
         year = year(date))


df_minmax <- df_timeserie %>% 
  group_by(name,year) %>% 
  reframe(min = min(mean),
          max = max(mean)) %>% 
  group_by(name) %>% 
  reframe(max = lm(max~year)$coefficients[[2]],
          min = lm(min~year)$coefficients[[2]], 
          diff = (max - min)/max)

mean(df_minmax$diff)

[1] 0.3825375

ℹ Please use `linewidth` instead.

3.5.0.
ℹ Please use the `legend.position.inside` argument of `theme()` instead.

Yearly SST time series for each of the 10 Mediterranean sub-regions are presented in <a href="#fig-MapMedSea_timeseries" class="quarto-xref">Figure 3</a>. Across all regions, periods of minimum and maximum temperatures occur synchronously, with minima in February–March and maxima in July–August. Overall, every sub-region exhibits higher SST values in 2024 compared to 1982. The most significant increase in summer temperature maxima is observed in Adriatic sea, with a warming rate of 0.63 °C per decade. Similarly, the most substantial increase in winter temperature minima occurs in Aegean Sea showing a warming rate of 0.4 °C per decade.

In [ ]:
knitr::include_graphics("Figs/plot_time_serie.png")

In [ ]:
library(patchwork)

df_date_extrem <- df3 %>%
  group_by(region, year) %>%
  summarise(
    doy_min_sst = yday[which.min(SST)],
    min_sst     = min(SST),
    doy_max_sst = yday[which.max(SST)],
    max_sst     = max(SST),
    .groups     = "drop"
  )
 

df_lm_extrem <- df_date_extrem %>% 
  dplyr::filter(doy_min_sst < 120) %>% 
  group_by(region) %>% 
  reframe(doy_min = lm(doy_min_sst ~ year)$coefficients[[2]],
          doy_max = lm(doy_max_sst ~ year)$coefficients[[2]])


# compute breaks every 15 days (1, 16, 31, … up to 361)
doy_breaks_15 <- seq(1, 366, by = 30)

# generate labels like “Jan 01”, “Jan 16”, …
doy_labels_15 <-  lubridate::month(as.Date(doy_breaks_15 - 1, origin = as.Date("2001-01-01")), label = TRUE, locale = "UTC") %>% 
  as.character()

min <- df_date_extrem %>% 
  ggplot(aes(x = year, y = doy_min_sst, color = region))+
  geom_point(alpha = 0.1, show.legend = F)+
  geom_smooth(method = "lm", alpha = 0.1, show.legend = F)+
  scale_color_manual(values = region_col)+
  ylim(0,120) +
  
  # labels
  labs(
    x        = "Year",
    y        = "Day of the year the minimum SST",
  ) +
  
  theme_minimal(base_size = 14) +
    
    scale_y_continuous(breaks = doy_breaks_15,
                       labels = doy_labels_15,
                       limits = c(0,120)) +
  theme(
    strip.background      = element_rect(fill = "gray90", color = NA),
    strip.text            = element_text(face = "bold"),
    
    # a little breathing room below panels
    # plot.margin           = margin(t = 5, r = 5, b = 30, l = 5),
    
    # grid tweaks
    panel.grid.major      = element_line(color = "gray85"),
    panel.grid.minor      = element_blank(),
    plot.background = element_rect(fill = "white", color = NA)
  )

Scale for y is already present.
Adding another scale for y, which will replace the existing scale.

Dates on which SST minima and maxima occur remain generally consistent over the study period for most regions (<a href="#fig-Med_minmax" class="quarto-xref">Figure 4</a>). However, shifts are observed in some areas: the winter temperature minimum in Levantine Sea occurs progressively earlier by 3.32 days per decade, while in Balearic Sea it occurs progressively later by 3.76 days per decade. Regarding the timing of maximum SST, two regions stand out: Adriatic sea and Aegean Sea both show an earlier occurrence of their annual temperature maxima by 2.41 and 1.58 days per decade, respectively.

In [ ]:
knitr::include_graphics("Figs/plot_min_max_SST.png")

In [ ]:
library(lubridate)
library(zoo)
library(trend) 
library(tidyverse)
library(data.table)
library(heatwaveR)



df <- fread("../Outputs/SST/region_stats.csv") %>% 
  mutate(date = as.character(date) %>% as.Date(format = "%Y%m%d"),
         mean = mean - 273.15) %>% 
  dplyr::select(date,region = "name",SST = "mean")

# Your input df must have columns:
#   date   (Date), region (chr/fct), SST (numeric)

# ── 1. Add year & day-of-year ──────────────────────────────────────────────
df2 <- df %>%
  mutate(
    year = year(date),
    yday = yday(date)
  )

# ── 2. Build climatology baseline (exclude most recent year) ───────────────
baseline <- df2 %>%
  filter(year <= max(year) - 1)

# ── 3. Compute region-specific warm-season threshold ───────────────────────
#     here using the 75th percentile of baseline SST per region
region_thresh <- baseline %>%
  group_by(region) %>%
  summarise(
    warm_thresh = quantile(SST, probs = 0.75, na.rm = TRUE),
    .groups = "drop"
  )


# ── 4. Join threshold back onto full data & compute anomalies ──────────────
df3 <- df2 %>%
  left_join(region_thresh, by = "region") %>%
  group_by(region, yday) %>%
  mutate(
    clim_mean = mean(SST[year <= max(year)-1], na.rm = TRUE),
    anomaly   = SST - clim_mean
  ) %>%
  ungroup()

# Inspect
# head(df3)

# ── 5. Threshold-based dates using region-specific warm_thresh ────────────
threshold_metrics <- df3 %>%
  group_by(region, year) %>%
  summarise(
    warm_onset = {
      r <- SST > warm_thresh
      dd <- rle(r)
      ends  <- cumsum(dd$lengths)
      starts<- ends - dd$lengths + 1
      idx   <- which(dd$values & dd$lengths >= 5)
      if(length(idx)) date[starts[idx[1]]] else as.Date(NA)
    },
    warm_end = {
      sel <- month(date) %in% 9:11
      r   <- sel & (SST < warm_thresh)
      dd  <- rle(r)
      ends  <- cumsum(dd$lengths)
      starts<- ends - dd$lengths + 1
      idx   <- which(dd$values & dd$lengths >= 5)
      if(length(idx)) date[starts[idx[1]]] else as.Date(NA)
    },
    .groups = "drop"
  )


lm_df_warm <- threshold_metrics %>% 
  mutate(doy_onset = yday(warm_onset),
         doy_offset = yday(warm_end), 
         length = doy_offset - doy_onset) %>% 
  group_by(region) %>% 
  reframe(rate_onset = lm(doy_onset ~ year)$coefficients[[2]],
          rate_end = lm(doy_offset ~ year)$coefficients[[2]],
          rate_length = lm(length ~ year)$coefficients[[2]])




# compute breaks every 15 days (1, 16, 31, … up to 361)
doy_breaks_15 <- seq(1, 366, by = 15)

# generate labels like “Jan 01”, “Jan 16”, …
doy_labels_15 <- format(
  as.Date(doy_breaks_15 - 1, origin = as.Date("2001-01-01")),
  "%b %d"
)


plot_doy_warm_start <- threshold_metrics %>%
  mutate(
    doy_onset  = yday(warm_onset),
    doy_offset = yday(warm_end),
    duration = doy_offset - doy_onset
  ) %>%
  select(-warm_onset, -warm_end) %>%
  pivot_longer(-c(region, year), names_to = "metric", values_to = "value") %>%
  mutate(metric = factor(metric, levels = c("doy_onset","doy_offset", "duration"),
                         labels = c("Warm Onset","Warm End","Duration"))) %>%
  dplyr::filter(metric == "Warm Onset") %>% 
  ggplot(aes(year, value, color = region)) +
  
    geom_point(alpha = 0.15, size = 1.5) +
    geom_smooth(method = "glm", se = TRUE, size = 0.8, alpha = 0.1) +
    
    facet_wrap(~ metric, ncol = 3, scales = "free_y") +
    
    scale_y_continuous(breaks = doy_breaks_15,
                       labels = doy_labels_15) +
    
    scale_color_manual(values = region_col, name = "Region") +
    
    # here’s the magic for label/key positions
    guides(color = guide_legend(
      direction      = "vertical",
      label.position = "left",   # move text left of the key
      key.position   = "right",  # move key to the right of the text
      label.hjust     = 0,       # left-align the text
      keywidth        = unit(1.2, "lines")
    )) +
    
    labs(x = "Year", y = "Day of Year") +
    
    theme_minimal(base_family = "Helvetica") +
    theme(
      # place the whole legend inside the plot:
      legend.position      = c(0.5, 0.98),
      legend.justification = c(0, 1),
      
      # semi-transparent background so it sits atop your points
      legend.background    = element_rect(fill = alpha("white", 0.6), color = NA),
      
      # strip out the old bottom legend settings:
      legend.direction     = NULL,
      legend.key.size      = unit(0.8, "lines"),
      legend.title         = element_text(size = 11, face = "bold"),
      legend.text          = element_text(size = 9),
      
      axis.title           = element_text(size = 12, face = "bold"),
      axis.text            = element_text(size = 10, color = "gray30"),
      strip.text           = element_text(size = 13, face = "bold"),
      panel.grid.major     = element_line(color = "gray90"),
      panel.grid.minor     = element_blank(),
      plot.background = element_rect(fill = "white",color = NA)
    )


plot_doy_warm_end <- threshold_metrics %>%
  mutate(
    doy_onset  = yday(warm_onset),
    doy_offset = yday(warm_end),
    duration = doy_offset - doy_onset
  ) %>%
  select(-warm_onset, -warm_end) %>%
  pivot_longer(-c(region, year), names_to = "metric", values_to = "value") %>%
  mutate(metric = factor(metric, levels = c("doy_onset","doy_offset", "duration"),
                         labels = c("Warm Onset","Warm End","Duration"))) %>%
  dplyr::filter(metric == "Warm End") %>% 
  ggplot(aes(year, value, color = region)) +
  
    geom_point(alpha = 0.15, size = 1.5, show.legend = F) +
    geom_smooth(method = "glm", se = TRUE, size = 0.8, show.legend = F, alpha = 0.1) +
    
    facet_wrap(~ metric, ncol = 3, scales = "free_y") +
    
    scale_y_continuous(breaks = doy_breaks_15,
                       labels = doy_labels_15) +
    
    scale_color_manual(values = region_col, name = "Region") +
    
    # here’s the magic for label/key positions
    guides(color = guide_legend(
      direction      = "vertical",
      label.position = "left",   # move text left of the key
      key.position   = "right",  # move key to the right of the text
      label.hjust     = 0,       # left-align the text
      keywidth        = unit(1.2, "lines")
    )) +
    
    labs(x = "Year", y = "Day of Year") +
    
    theme_minimal(base_family = "Helvetica") +
    theme(
      # place the whole legend inside the plot:
      # legend.position      = c(0.3, 0.98),
      legend.justification = c(0, 1),
      
      # semi-transparent background so it sits atop your points
      legend.background    = element_rect(fill = alpha("white", 0.6), color = NA),
      
      # strip out the old bottom legend settings:
      legend.direction     = NULL,
      legend.key.size      = unit(0.8, "lines"),
      legend.title         = element_text(size = 11, face = "bold"),
      legend.text          = element_text(size = 9),
      
      axis.title           = element_text(size = 12, face = "bold"),
      axis.text            = element_text(size = 10, color = "gray30"),
      strip.text           = element_text(size = 13, face = "bold"),
      panel.grid.major     = element_line(color = "gray90"),
      panel.grid.minor     = element_blank(),
      plot.background = element_rect(fill = "white",color = NA)
    )


plot_doy_warm_duration <- threshold_metrics %>%
  mutate(
    doy_onset  = yday(warm_onset),
    doy_offset = yday(warm_end),
    duration = doy_offset - doy_onset
  ) %>%
  select(-warm_onset, -warm_end) %>%
  pivot_longer(-c(region, year), names_to = "metric", values_to = "value") %>%
  mutate(metric = factor(metric, levels = c("doy_onset","doy_offset", "duration"),
                         labels = c("Warm Onset","Warm End","Duration of warm period"))) %>%
  dplyr::filter(metric == "Duration of warm period") %>% 
  ggplot(aes(year, value, color = region)) +
  
    geom_point(alpha = 0.15, size = 1.5, show.legend = F) +
    geom_smooth(method = "glm", se = TRUE, size = 0.8, show.legend = F, alpha = 0.1) +
    
    facet_wrap(~ metric, ncol = 3, scales = "free_y") +
    
    scale_color_manual(values = region_col, name = "Region") +
    
    # here’s the magic for label/key positions
    guides(color = guide_legend(
      direction      = "vertical",
      label.position = "left",   # move text left of the key
      key.position   = "right",  # move key to the right of the text
      label.hjust     = 0,       # left-align the text
      keywidth        = unit(1.2, "lines")
    )) +
    
    labs(x = "Year", y = "Number of Days") +
    
    theme_minimal(base_family = "Helvetica") +
    theme(
      # place the whole legend inside the plot:
      # legend.position      = c(0.3, 0.98),
      legend.justification = c(0, 1),
      
      # semi-transparent background so it sits atop your points
      legend.background    = element_rect(fill = alpha("white", 0.6), color = NA),
      
      # strip out the old bottom legend settings:
      legend.direction     = NULL,
      legend.key.size      = unit(0.8, "lines"),
      legend.title         = element_text(size = 11, face = "bold"),
      legend.text          = element_text(size = 9),
      
      axis.title           = element_text(size = 12, face = "bold"),
      axis.text            = element_text(size = 10, color = "gray30"),
      strip.text           = element_text(size = 13, face = "bold"),
      panel.grid.major     = element_line(color = "gray90"),
      panel.grid.minor     = element_blank(),
      plot.background = element_rect(fill = "white",color = NA)
    )


plot_doy_warm_merged <- (plot_doy_warm_start + plot_doy_warm_end) + plot_doy_warm_duration
# ggsave("Manuscript/Figs/plot_warm_season.png",plot_doy_warm_merged, dpi = 300)

<a href="#fig-Med_warm_metrics" class="quarto-xref">Figure 5</a> (left) shows the start date of the warm season in all subregions of the Mediterranean Sea. The warm season begins when the daily SST exceeds the 75th percentile (p75) of the subregion’s SST for five consecutive days. A downward trend is evident across all subregions, indicating that the warm season now starts on average 5.66 days earlier per decade. The Levantine Sea shows the greatest advancement in onset, at 8.23 days per decade.

Regarding the end of the warm season, a consistent delay is observed across all subregions, with the season ending on average 3.69 days later per decade (<a href="#fig-Med_warm_metrics" class="quarto-xref">Figure 5</a> middle). Two regions stand out—Levantine Sea and Aegean Sea— showing shifts of 7.01 and 5.33 days per decade, respectively.

<a href="#fig-Med_warm_metrics" class="quarto-xref">Figure 5</a> (right) shows the length of the warm season, defined as the interval between its end and start dates. Across the Mediterranean basin, the warm season is lengthening by 9.35 days per decade on average. All subregions exhibit this trend, except for Levantine Sea and Aegean Sea, which show increases of 15.24 and 11.48 days per decade, respectively.

In [ ]:
knitr::include_graphics("Figs/plot_warm_season.png")

In [ ]:
library(lubridate)
library(zoo)
library(trend) 
library(tidyverse)
library(data.table)
library(heatwaveR)

# ── 8. Seasonal-cycle stats ────────────────────────────────────────────────
seasonal_metrics <- df3 %>%
  group_by(region, year) %>%
  summarise(
    date_max     = date[which.max(SST)],
    date_min     = date[which.min(SST)],
    amplitude    = max(SST) - min(SST),
    warming_rate = {
      sel <- month(date) %in% 3:5
      if(sum(sel) > 2) coef(lm(SST[sel] ~ yday[sel]))[2] else NA
    },
    cooling_rate = {
      sel <- month(date) %in% 9:11
      if(sum(sel) > 2) coef(lm(SST[sel] ~ yday[sel]))[2] else NA
    },
    .groups = "drop"
  )


lm_df_amp <- seasonal_metrics  %>% 
  group_by(region) %>% 
  reframe(rate_amplitude = lm(amplitude ~ year)$coefficients[[2]])

df_amp <- seasonal_metrics %>% 
  group_by(region) %>% 
  reframe(mean = mean(amplitude, na.rm = T))


plot_amplitude <- seasonal_meanplot_amplitude <- seasonal_metrics %>% 
  ggplot(aes(x = year, y = amplitude, color = region))+
  geom_point(alpha = 0.3)+
  geom_smooth(method = "lm",se = T, alpha = 0.1)+
  scale_color_manual(values = region_col, name = "Region")+
    
    labs(x = "Year", y = "Thermal amplitude (°C)") +
    
    theme_minimal(base_family = "Helvetica") +
    theme(
      # place the whole legend inside the plot:
      # legend.position      = c(0.3, 0.98),
      legend.justification = c(0, 1),
      
      # semi-transparent background so it sits atop your points
      legend.background    = element_rect(fill = alpha("white", 0.6), color = NA),
      
      # strip out the old bottom legend settings:
      legend.direction     = NULL,
      legend.key.size      = unit(0.8, "lines"),
      legend.title         = element_text(size = 11, face = "bold"),
      legend.text          = element_text(size = 9),
      legend.position = "bottom",
      
      axis.title           = element_text(size = 12, face = "bold"),
      axis.text            = element_text(size = 10, color = "gray30"),
      strip.text           = element_text(size = 13, face = "bold"),
      panel.grid.major     = element_line(color = "gray90"),
      panel.grid.minor     = element_blank(),
      plot.background = element_rect(fill = "white",color = NA)
    )

# ggsave("Manuscript/Figs/plot_amplitude.png",plot_amplitude, dpi = 300)

The thermic amplitude is defined as the difference between the minimum winter SST and the maximum summer SST. The subregion with the smallest thermal amplitude is Alboran Sea, averaging just 10.82°C between summer peaks and winter troughs. By contrast, Adriatic sea exhibits the largest thermal amplitude in the Mediterranean Sea, at 14.94. Across the entire basin, there is an increasing trend in thermic amplitude, averaging 0.17°C per decade. The region with the most pronounced rise is Adriatic sea where amplitude grows by 0.32°C each decade. This divergence stems from summer temperatures rising about 38 % faster than winter temperatures across the basin.

In [ ]:
knitr::include_graphics("Figs/plot_amplitude.png")

## 3.2 Heatwaves by regions

In [ ]:
library(tidyverse)
library(heatwaveR)

df_timeserie <- fread("../Outputs/SST/region_stats.csv") %>% 
  mutate(date = as.character(date) %>% as.Date(format = "%Y%m%d"),
         mean = mean - 273.15,
         median = median - 273.15,
         p10 = p10 - 273.15,
         p25 = p25 - 273.15,
         p75 = p75 - 273.15,
         p90 = p90 - 273.15) %>%
  as_tibble() %>% 
  mutate(doy = yday(date),
         year = year(date))

df_timeserie %>% 
  dplyr::filter(name == "Adriatic sea") %>% 
  ts2clm(date,mean)

## 3.3 Heatwaves Pixel per Pixels

### 3.3.1 Number of Event per year

In [ ]:
library(sf)
library(ggplot2)
library(rnaturalearth)
library(rnaturalearthdata)
library(tidyverse)
library(terra)
library(viridisLite)
library(ggspatial)
library(Utilities.Package)


region_col <- c(
  # greens
  "Levantine Sea" = "#008F1A",   # brighter pine-green
  "Sea of Sicily" = "#6B9C3C",   # vivid olive-sage
  
  # blues / teals / purples
  "Alboran Sea"   = "#414E7C",   # richer slate-blue
  "Gulf of Lyon"  = "#266A78",   # clearer Mediterranean teal
  "Adriatic sea"  = "#583C68",   # deeper indigo-violet
  
  # yellows / ochres
  "Balearic Sea"  = "#A98C3D",   # crisp mustard-ochre
  "Aegean Sea"    = "#D7A543",   # sun-baked golden-ochre
  
  # reds / oranges / browns
  "Tyrrhenian Sea"= "#A02A2A",   # vivid brick-red
  "Algerian Sea"  = "#DF5513",   # bright burnt-orange
  "Ionian Sea"    = "#705943"    # warmer earthy-taupe
)
# Load your shapefile (replace with your path)
med_regions <- st_read("Data/mask_Med_Sea/More_regions_Med_Sea.shp") %>% 
  dplyr::filter(!str_detect(name, "Black"))

# Get world land polygons
world <- ne_countries(scale = "large", returnclass = "sf")


filelist <- "Outputs/SST/nc/n_event_per_year" %>% 
  list.files(full.names = T) %>% 
  as_tibble() %>% 
  rename(path = value) %>% 
  mutate(file = gsub(".*/","",path),
         year = substr(file,9,12))


stk <- filelist$path %>% 
  rast()
names(stk) <- filelist$year

stk_avg <- mean(stk, na.rm = T)

map_n_event <- ggplot() +
  ## ── data layers ─────────────────────────────────────────────
  tidyterra::geom_spatraster(data = stk_avg,
                             aes(fill = mean)) + 
  geom_sf(data = world, fill = "grey80", color = NA) +
  ## ── colour scales ───────────────────────────────────────────
  scale_fill_gradientn(
    colours  = turbo(10),
    na.value = "transparent",
    name     = "Avg number of Event per Year"
  ) +
    ## ── horizontal guide with title on top ──────────────────────
  guides(
    fill = guide_colourbar(
      direction      = "horizontal",
      title.position = "top",
      label.position = "bottom",            # optional – labels below bar
      barwidth       = unit(5, "cm"),       # tweak length
      barheight      = unit(0.45, "cm")     # tweak thickness
    )
  ) +
  ## ── map extent ──────────────────────────────────────────────
  coord_sf(xlim = c(-10, 40), ylim = c(30, 47), expand = FALSE) +
  ## ── *** scale bar *** ───────────────────────────────────────
  annotation_scale(
    location   = "bl",   # bottom-left
    width_hint = 0.25,   # ≈25 % of the map width
    line_width = 0.6,
    text_cex   = 2,
    pad_x      = unit(0.4, "cm"),
    pad_y      = unit(0.4, "cm")
  ) +
  ## ── theme ───────────────────────────────────────────────────
  theme_minimal(base_size = 18) +
  theme(
    panel.grid        = element_line(color = "gray90"),
    panel.background  = element_rect(fill = "aliceblue"),
    plot.background   = element_rect(fill = "white", color = NA),

    # move legend INSIDE, upper-left; anchor by its top-left corner
    legend.position       = c(0.02, 0.98),   # npc coords (x = 2 %, y = 98 %)
    legend.justification  = c(0, 1),
    legend.direction      = "horizontal",

    legend.title.align    = 0.5,             # centre title
    legend.text           = element_text(size = 13),

    # give legend a white background & a bit of padding
    legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA,linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4)
  )

ggsave("Manuscript/Figs/Map_MedSea_n_event.png", map_n_event, dpi = 300, height =575*3, width = 1319*3, units = "px")

############ Line plot by region
med_regions <- st_transform(med_regions, crs(stk))

# med_regions <- st_zm(med_regions)

zone_rast <- terra::rasterize(med_regions, stk[[1]], field = 1:nrow(med_regions))
writeRaster(zone_rast, "Data/mask_Med_Sea/region_of_the_med.tif")
plot(zone_rast)
# Function that terra::zonal will apply: returns a 3-element vector
qfun <- function(values, ...) {
  mean(values, na.rm = TRUE)
}

# zonal() summarises **all layers at once**
z_raw <- terra::zonal(stk, zone_rast, fun = qfun)

z_df <- as.data.frame(z_raw)  %>% 
  mutate(region = med_regions$name) %>% 
  dplyr::select(-layer) %>% # attach names
  pivot_longer(c(-region),
               names_to   = "year",
               values_to = "mean") %>% 
  mutate(year = as.numeric(year))


plot_lines <- ggplot(z_df, aes(x = year, y = mean, colour = region)) +
  geom_point(alpha = 0.2, show.legend = F) +
  geom_line(alpha = 0.2, show.legend = F) +
  scale_colour_manual(values = region_col) +

  geom_smooth(
    method    = "gam",
    linewidth = 1.2,
    alpha     = 0.05,
    formula     = (y) ~ s(x, k = 3),  # <-- k goes here
    show.legend = F) +

  ## ── 1. force a ONE-column legend ────────────────────────────
  guides(
    colour = guide_legend(ncol = 1)   # 1 column, items stacked
  ) +
  ylim(c(0,11.25))+

  labs(colour = "Regions", x = "Year", y = "Average number of HW event") +

  ## ── 2. theme tweaks to pin legend inside, top-left ──────────
  theme_minimal(base_size = 18) +
  theme(
    plot.background   = element_rect(fill = "white", colour = NA),
    legend.text       = element_text(size = 13),

    legend.position       = c(0.02, 0.98),   # (x, y) in npc units
    legend.justification  = c(0, 1),         # anchor by its top-left corner
    legend.direction      = "vertical",      # keeps items vertical
    legend.background     = element_rect(fill = "white", colour = NA, linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4),

    legend.key.width      = unit(2, "cm")    # leave if you want wider keys
  )

plot_lines

ggsave("Manuscript/Figs/line_n_event.png", plot_lines, dpi = 300, height =575*3, width = 1319*3, units = "px")
###################### MAP of TREND

df_map_freq_long <- stk %>%
  as.data.frame(xy = T) %>% 
  mutate(ID = c(1:nrow(.))) %>% 
  pivot_longer(-c(ID,x,y), values_to = "value", names_to = "year") %>% 
  mutate(value = case_when(is.na(value) ~ 0,
                           T ~ value)) %>% 
  mutate(year = as.numeric(year)) %>% 
  group_by(ID) %>%
  summarise(trend = coef(lm(value ~ year))[2])

df_freq <- stk %>%
  as.data.frame(xy = T) %>%  
  mutate(ID = c(1:nrow(.))) %>% 
  dplyr::select(c(x,y,ID))


df_trend <- df_map_freq_long %>% 
  # dplyr::filter(trend < quantile(trend, probs = 0.95)) %>% 
  left_join(df_freq, by = "ID") %>% 
  dplyr::select(-ID) %>% 
  relocate(x,y,trend)

rast_trend <- stk$`1982`
values(rast_trend) <- NA

pts <- vect(df_trend, geom = c("x", "y"), crs = crs(rast_trend))

rast_trend <- terra::rasterize(
  pts,             # what to burn
  rast_trend,         # target raster geometry
  field = "trend", # column to transfer
  fun   = "mean"   # how to handle duplicate points (mean, max, sum, …)
)

# Function that terra::zonal will apply: returns a 3-element vector
qfun <- function(values, ...) {
  mean(values, na.rm = TRUE)
}


# zonal() summarises **all layers at once**
z_raw_trend <- terra::zonal(rast_trend, zone_rast, fun = qfun)

# 1.  Add centroid geometry and its coordinates to `med_regions`
med_regions <- med_regions %>%                                 # already an sf multipolygon
  mutate(centroid = st_centroid(geometry),                     # geometry column is usually called `geometry`
         lon      = st_coordinates(centroid)[, 1],             # X
         lat      = st_coordinates(centroid)[, 2])             # Y
#   └─> use st_point_on_surface(geometry) instead of st_centroid() if you
#       must ensure the point falls inside every (multi)polygon

# 2.  Keep only what you want to join
centroids_tbl <- med_regions %>% 
  st_drop_geometry() %>%                                       # drops *all* sf geometries
  select(name, lon, lat)  

z_df_trend <- as.data.frame(z_raw_trend)  %>% 
  mutate(region = med_regions$name) %>% 
  dplyr::select(-layer) %>%
  mutate(mean = round(mean,3)) %>% 
  left_join(centroids_tbl, by = c("region" = "name"))


map_trend <- df_map_freq_long %>% 
  left_join(df_freq, by = "ID") %>% 
  ggplot() +
    ## ── data layers ───────────────────────────────────────────
    geom_tile(aes(x = x, y = y, fill = trend)) +
    geom_sf(data = world, fill = "grey80", colour = NA) +

    ## ── colour scale ──────────────────────────────────────────
    tidyterra::scale_fill_grass_c(
      name = "Trends of HW frequency (Event per year)",
      use_grass_range = FALSE
    ) +

    ## ── 1. horizontal colour bar, title on top ───────────────
    guides(
      fill = guide_colourbar(
        direction      = "horizontal",
        title.position = "top",
        label.position = "bottom",            # labels under the bar (optional)
        barwidth       = unit(10, "cm"),       # adjust length
        barheight      = unit(0.45, "cm")     # adjust thickness
      )
    ) +

    ## ── map extent ───────────────────────────────────────────
    coord_sf(xlim = c(-10, 40), ylim = c(30, 47), expand = FALSE) +

    ## ── region means & colours ───────────────────────────────
    geom_label(data = z_df_trend,
              aes(x = lon, y = lat, label = mean, colour = region),
              show.legend = FALSE, fill = rgb(1,1,1,0.7), size = 5) +
    scale_colour_manual(values = region_col, name = "") +

    ## ── scale bar ────────────────────────────────────────────
    annotation_scale(
      location   = "bl",
      width_hint = 0.25,
      line_width = 0.6,
      text_cex   = 2,
      pad_x      = unit(0.4, "cm"),
      pad_y      = unit(0.4, "cm")
    ) +

    ## ── theme ────────────────────────────────────────────────
    theme_minimal(base_size = 18) +
    theme(
      panel.grid        = element_line(colour = "gray90"),
      panel.background  = element_rect(fill = "aliceblue"),
      plot.background   = element_rect(fill = "white", colour = NA),

      # 2. park legend INSIDE, top-left
      legend.position       = c(0.02, 0.98),  # npc coords: 2 % from left, 98 % up
      legend.justification  = c(0, 1),        # anchor by its top-left corner
      legend.direction      = "horizontal",

      legend.text           = element_text(size = 13),
      legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA, linewidth = 0.2),
      legend.margin         = margin(4, 4, 4, 4),

      axis.title            = element_blank()
    )

map_trend

ggsave("Manuscript/Figs/trend_frequency.png", map_trend, dpi = 300, height =575*3, width = 1319*3, units = "px")


### BOXPLOT

# ── libraries ───────────────────────────────────────────────────────────
library(terra)   # raster/terra objects
library(sf)      # modern vector data
library(ggplot2) # plotting
library(dplyr)   # pipe-friendly helpers (optional)

# ── 1. load data ────────────────────────────────────────────────────────
r   <- rast_trend         # single- or multi-band
reg <- med_regions           # polygons

# Make sure both layers share the same CRS
reg <- st_transform(reg, crs(r))

# ── 2. extract values per polygon ───────────────────────────────────────
# terra::extract() returns a data.frame: one row per cell per polygon
#   ID   = polygon index (generated automatically)
#   lyr1 = raster value (one column per raster layer)
#
vals <- terra::extract(r, vect(reg))            # vect() converts sf → SpatVector

# ── 3. tidy up: attach region names / attributes ───────────────────────
# Suppose your regions layer has a column called "NAME"
ID_name <- reg %>% 
      as_tibble() %>%  
      dplyr::mutate(ID = c(1:10),
                    name = as_factor(name))  %>%       # ensure matching ID
      dplyr::select(ID, name)

vals <- vals %>% 
  dplyr::left_join(ID_name,by = "ID") %>% 
  dplyr::filter(mean != 0)%>%                       # your summary data-frame
  mutate(
    name = fct_reorder(name, mean, .desc = TRUE)  # order by mean, descending
  )

# ── 4. plot ─────────────────────────────────────────────────────────────
plot_boxplot <- ggplot() +
  # 1. data layer (legend suppressed)
  geom_boxplot(
    data          = vals,
    aes(x = name, y = mean, fill = name),
    colour        = "black",
    linewidth     = 0.7,
    # size = 2,
    # alpha         = .60,
    outlier.shape = 21,
    outlier.fill  = NA,
    show.legend   = FALSE
  ) +

  # 2. dummy layer that only feeds the legend (square keys)
  geom_point(
    data = tibble(name = names(region_col), x = NA_real_, y = NA_real_),
    aes(x, y, fill = name),
    shape        = 22,        # solid square
    size         = 6,
    stroke       = 0,
    show.legend  = TRUE,
    inherit.aes  = FALSE
  )+

  ## ---- scales -----------------------------------------------------------
  scale_fill_manual(values = region_col, name = "Region") +
  scale_colour_manual(values = region_col, guide = "none") +

  guides(fill = guide_legend(
    nrow       = 2,                 # 1 column ⇒ 4 rows
    byrow      = TRUE,
    keywidth   = unit(0.6, "cm"),   # square keys
    keyheight  = unit(0.6, "cm")
  )) +
  ## ---- labels & theme ---------------------------------------------------
  labs(
    x = NULL,
    y = "Trends of HW frequency (Event per year)"
  ) +
  theme_minimal(base_size = 18) +
  theme(
    plot.background   = element_rect(fill = "white", colour = NA),
    legend.text       = element_text(size = 13),

    legend.position       = c(0.02, 0.15),   # (x, y) in npc units
    legend.justification  = c(0, 1),         # anchor by its top-left corner
    legend.direction      = "vertical",      # keeps items vertical
    legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA, linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4),

    legend.key.width      = unit(2, "cm"),    # leave if you want wider keys
    axis.text.x = element_blank()
  )+
  ylim(c(0,0.23))

plot_boxplot

ggsave("Manuscript/Figs/trend_boxplot.png", plot_boxplot, dpi = 300, height =575*3, width = 1319*3, units = "px")

In [ ]:
library(magick)

# ── 1. read the four individual plots ───────────────────────────────────
img_a <- image_read("Manuscript/Figs/Map_MedSea_n_event.png")
img_b <- image_read("Manuscript/Figs/line_n_event.png")
img_c <- image_read("Manuscript/Figs/trend_frequency.png")
img_d <- image_read("Manuscript/Figs/trend_boxplot.png")

# ── 2. add a corner label to each plot ──────────────────────────────────
stamp <- function(img, letter,
                  size = 80,               # point size of the label
                  gravity = "northwest",   # top-left corner
                  offset  = "+20+15") {    # x,y offset from the corner
  image_annotate(
    img, letter,
    font      = "Helvetica-Bold",
    size      = size,
    gravity   = gravity,
    location  = offset,
    color     = "black",
    boxcolor  = "white",   # gives a small white rectangle behind the letter
    weight    = 700        # ensures the stroke is bold
  )
}

img_a <- stamp(img_a, "A")
img_b <- stamp(img_b, "B")
img_c <- stamp(img_c, "C")
img_d <- stamp(img_d, "D")

# ── 3. assemble the composite exactly as before ─────────────────────────
row1  <- image_append(c(img_a, img_b), stack = FALSE)  # side-by-side
row2  <- image_append(c(img_c, img_d), stack = FALSE)

combo <- image_append(c(row1, row2),   stack = TRUE)   # one above the other

image_write(combo, path = "Manuscript/Figs/plot_n_events.png")

### 3.3.2 Intensity

In [ ]:
library(sf)
library(ggplot2)
library(rnaturalearth)
library(rnaturalearthdata)
library(tidyverse)
library(terra)
library(viridisLite)
library(ggspatial)
library(Utilities.Package)


region_col <- c(
  # greens
  "Levantine Sea" = "#008F1A",   # brighter pine-green
  "Sea of Sicily" = "#6B9C3C",   # vivid olive-sage
  
  # blues / teals / purples
  "Alboran Sea"   = "#414E7C",   # richer slate-blue
  "Gulf of Lyon"  = "#266A78",   # clearer Mediterranean teal
  "Adriatic sea"  = "#583C68",   # deeper indigo-violet
  
  # yellows / ochres
  "Balearic Sea"  = "#A98C3D",   # crisp mustard-ochre
  "Aegean Sea"    = "#D7A543",   # sun-baked golden-ochre
  
  # reds / oranges / browns
  "Tyrrhenian Sea"= "#A02A2A",   # vivid brick-red
  "Algerian Sea"  = "#DF5513",   # bright burnt-orange
  "Ionian Sea"    = "#705943"    # warmer earthy-taupe
)
# Load your shapefile (replace with your path)
med_regions <- st_read("Data/mask_Med_Sea/More_regions_Med_Sea.shp") %>% 
  dplyr::filter(!str_detect(name, "Black"))

# Get world land polygons
world <- ne_countries(scale = "large", returnclass = "sf")


filelist <- "Outputs/SST/nc/intensity_mean" %>% 
  list.files(full.names = T) %>% 
  as_tibble() %>% 
  rename(path = value) %>% 
  mutate(file = gsub(".*/","",path),
         year = substr(file,16,19))


stk <- filelist$path %>% 
  rast()
names(stk) <- filelist$year

stk_avg <- mean(stk, na.rm = T)

map_intensity <- ggplot() +
  ## ── data layers ─────────────────────────────────────────────
  tidyterra::geom_spatraster(data = stk_avg,
                             aes(fill = mean)) + 
  geom_sf(data = world, fill = "grey80", color = NA) +
  ## ── colour scales ───────────────────────────────────────────
  scale_fill_gradientn(
    colours  = turbo(10),
    na.value = "transparent",
    name     = "Mean intensity of HW events (°C)"
  ) +
    ## ── horizontal guide with title on top ──────────────────────
  guides(
    fill = guide_colourbar(
      direction      = "horizontal",
      title.position = "top",
      label.position = "bottom",            # optional – labels below bar
      barwidth       = unit(5, "cm"),       # tweak length
      barheight      = unit(0.45, "cm")     # tweak thickness
    )
  ) +
  ## ── map extent ──────────────────────────────────────────────
  coord_sf(xlim = c(-10, 40), ylim = c(30, 47), expand = FALSE) +
  ## ── *** scale bar *** ───────────────────────────────────────
  annotation_scale(
    location   = "bl",   # bottom-left
    width_hint = 0.25,   # ≈25 % of the map width
    line_width = 0.6,
    text_cex   = 2,
    pad_x      = unit(0.4, "cm"),
    pad_y      = unit(0.4, "cm")
  ) +
  ## ── theme ───────────────────────────────────────────────────
  theme_minimal(base_size = 18) +
  theme(
    panel.grid        = element_line(color = "gray90"),
    panel.background  = element_rect(fill = "aliceblue"),
    plot.background   = element_rect(fill = "white", color = NA),

    # move legend INSIDE, upper-left; anchor by its top-left corner
    legend.position       = c(0.02, 0.98),   # npc coords (x = 2 %, y = 98 %)
    legend.justification  = c(0, 1),
    legend.direction      = "horizontal",

    legend.title.align    = 0.5,             # centre title
    legend.text           = element_text(size = 13),

    # give legend a white background & a bit of padding
    legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA,linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4)
  )

map_intensity

ggsave("Manuscript/Figs/Map_MedSea_intensity.png", map_intensity, dpi = 300, height =575*3, width = 1319*3, units = "px")

############ Line plot by region
med_regions <- st_transform(med_regions, crs(stk))

# med_regions <- st_zm(med_regions)

zone_rast <- terra::rasterize(med_regions, stk[[1]], field = 1:nrow(med_regions))
plot(zone_rast)
# Function that terra::zonal will apply: returns a 3-element vector
qfun <- function(values, ...) {
  mean(values, na.rm = TRUE)
}

# zonal() summarises **all layers at once**
z_raw <- terra::zonal(stk, zone_rast, fun = qfun)

z_df <- as.data.frame(z_raw)  %>% 
  mutate(region = med_regions$name) %>% 
  dplyr::select(-layer) %>% # attach names
  pivot_longer(c(-region),
               names_to   = "year",
               values_to = "mean") %>% 
  mutate(year = as.numeric(year))


plot_lines <- ggplot(z_df, aes(x = year, y = mean, colour = region)) +
  geom_point(alpha = 0.2, show.legend = F) +
  geom_line(alpha = 0.2, show.legend = F) +
  scale_colour_manual(values = region_col) +

  geom_smooth(
    method    = "glm",
    linewidth = 1.2,
    alpha     = 0.05,
    # formula     = (y) ~ s(x),  # <-- k goes here
    show.legend = F) +

  ## ── 1. force a ONE-column legend ────────────────────────────
  guides(
    colour = guide_legend(ncol = 1)   # 1 column, items stacked
  ) +
  # ylim(c(0,11.25))+

  labs(colour = "Regions", x = "Year", y = "Mean intensity of HW event (°C)") +

  ## ── 2. theme tweaks to pin legend inside, top-left ──────────
  theme_minimal(base_size = 18) +
  theme(
    plot.background   = element_rect(fill = "white", colour = NA),
    legend.text       = element_text(size = 13),

    legend.position       = c(0.02, 0.98),   # (x, y) in npc units
    legend.justification  = c(0, 1),         # anchor by its top-left corner
    legend.direction      = "vertical",      # keeps items vertical
    legend.background     = element_rect(fill = "white", colour = NA, linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4),

    legend.key.width      = unit(2, "cm")    # leave if you want wider keys
  )

plot_lines

ggsave("Manuscript/Figs/line_intensity.png", plot_lines, dpi = 300, height =575*3, width = 1319*3, units = "px")
###################### MAP of TREND

df_map_intensity_long <- stk %>%
  as.data.frame(xy = T) %>% 
  mutate(ID = c(1:nrow(.))) %>% 
  pivot_longer(-c(ID,x,y), values_to = "value", names_to = "year") %>% 
  mutate(value = case_when(is.na(value) ~ 0,
                           T ~ value)) %>% 
  mutate(year = as.numeric(year)) %>% 
  group_by(ID) %>%
  summarise(trend = coef(lm(value ~ year))[2])

df_intensity <- stk %>%
  as.data.frame(xy = T) %>%  
  mutate(ID = c(1:nrow(.))) %>% 
  dplyr::select(c(x,y,ID))


df_trend <- df_map_intensity_long %>% 
  # dplyr::filter(trend < quantile(trend, probs = 0.95)) %>% 
  left_join(df_intensity, by = "ID") %>% 
  dplyr::select(-ID) %>% 
  relocate(x,y,trend)

rast_trend <- stk$`1982`
values(rast_trend) <- NA

pts <- vect(df_trend, geom = c("x", "y"), crs = crs(rast_trend))

rast_trend <- terra::rasterize(
  pts,             # what to burn
  rast_trend,         # target raster geometry
  field = "trend", # column to transfer
  fun   = "mean"   # how to handle duplicate points (mean, max, sum, …)
)

# Function that terra::zonal will apply: returns a 3-element vector
qfun <- function(values, ...) {
  mean(values, na.rm = TRUE)
}


# zonal() summarises **all layers at once**
z_raw_trend <- terra::zonal(rast_trend, zone_rast, fun = qfun)

# 1.  Add centroid geometry and its coordinates to `med_regions`
med_regions <- med_regions %>%                                 # already an sf multipolygon
  mutate(centroid = st_centroid(geometry),                     # geometry column is usually called `geometry`
         lon      = st_coordinates(centroid)[, 1],             # X
         lat      = st_coordinates(centroid)[, 2])             # Y
#   └─> use st_point_on_surface(geometry) instead of st_centroid() if you
#       must ensure the point falls inside every (multi)polygon

# 2.  Keep only what you want to join
centroids_tbl <- med_regions %>% 
  st_drop_geometry() %>%                                       # drops *all* sf geometries
  select(name, lon, lat)  

z_df_trend <- as.data.frame(z_raw_trend)  %>% 
  mutate(region = med_regions$name) %>% 
  dplyr::select(-layer) %>%
  mutate(mean = round(mean,3)) %>% 
  left_join(centroids_tbl, by = c("region" = "name"))


map_trend <- df_map_intensity_long %>% 
  left_join(df_intensity, by = "ID") %>% 
  ggplot() +
    ## ── data layers ───────────────────────────────────────────
    geom_tile(aes(x = x, y = y, fill = trend)) +
    geom_sf(data = world, fill = "grey80", colour = NA) +

    ## ── colour scale ──────────────────────────────────────────
    tidyterra::scale_fill_grass_c(
      name = "Trends of HW intensity (°C per year)",
      use_grass_range = FALSE
    ) +

    ## ── 1. horizontal colour bar, title on top ───────────────
    guides(
      fill = guide_colourbar(
        direction      = "horizontal",
        title.position = "top",
        label.position = "bottom",            # labels under the bar (optional)
        barwidth       = unit(10, "cm"),       # adjust length
        barheight      = unit(0.45, "cm")     # adjust thickness
      )
    ) +

    ## ── map extent ───────────────────────────────────────────
    coord_sf(xlim = c(-10, 40), ylim = c(30, 47), expand = FALSE) +

    ## ── region means & colours ───────────────────────────────
    geom_label(data = z_df_trend,
              aes(x = lon, y = lat, label = mean, colour = region),
              show.legend = FALSE, fill = rgb(1,1,1,0.7), size = 5) +
    scale_colour_manual(values = region_col, name = "") +

    ## ── scale bar ────────────────────────────────────────────
    annotation_scale(
      location   = "bl",
      width_hint = 0.25,
      line_width = 0.6,
      text_cex   = 2,
      pad_x      = unit(0.4, "cm"),
      pad_y      = unit(0.4, "cm")
    ) +

    ## ── theme ────────────────────────────────────────────────
    theme_minimal(base_size = 18) +
    theme(
      panel.grid        = element_line(colour = "gray90"),
      panel.background  = element_rect(fill = "aliceblue"),
      plot.background   = element_rect(fill = "white", colour = NA),

      # 2. park legend INSIDE, top-left
      legend.position       = c(0.02, 0.98),  # npc coords: 2 % from left, 98 % up
      legend.justification  = c(0, 1),        # anchor by its top-left corner
      legend.direction      = "horizontal",

      legend.text           = element_text(size = 13),
      legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA, linewidth = 0.2),
      legend.margin         = margin(4, 4, 4, 4),

      axis.title            = element_blank()
    )

map_trend

ggsave("Manuscript/Figs/trend_intensity.png", map_trend, dpi = 300, height =575*3, width = 1319*3, units = "px")


### BOXPLOT

# ── libraries ───────────────────────────────────────────────────────────
library(terra)   # raster/terra objects
library(sf)      # modern vector data
library(ggplot2) # plotting
library(dplyr)   # pipe-friendly helpers (optional)

# ── 1. load data ────────────────────────────────────────────────────────
r   <- rast_trend         # single- or multi-band
reg <- med_regions           # polygons

# Make sure both layers share the same CRS
reg <- st_transform(reg, crs(r))

# ── 2. extract values per polygon ───────────────────────────────────────
# terra::extract() returns a data.frame: one row per cell per polygon
#   ID   = polygon index (generated automatically)
#   lyr1 = raster value (one column per raster layer)
#
vals <- terra::extract(r, vect(reg))            # vect() converts sf → SpatVector

# ── 3. tidy up: attach region names / attributes ───────────────────────
# Suppose your regions layer has a column called "NAME"
ID_name <- reg %>% 
      as_tibble() %>%  
      dplyr::mutate(ID = c(1:10),
                    name = as_factor(name))  %>%       # ensure matching ID
      dplyr::select(ID, name)

vals <- vals %>% 
  dplyr::left_join(ID_name,by = "ID") %>% 
  dplyr::filter(mean != 0)%>%                       # your summary data-frame
  mutate(
    name = fct_reorder(name, mean, .desc = TRUE)  # order by mean, descending
  )

# ── 4. plot ─────────────────────────────────────────────────────────────
plot_boxplot <- ggplot() +
  # 1. data layer (legend suppressed)
  geom_boxplot(
    data          = vals,
    aes(x = name, y = mean, fill = name),
    colour        = "black",
    linewidth     = 0.7,
    # size = 2,
    # alpha         = .60,
    outlier.shape = 21,
    outlier.fill  = NA,
    show.legend   = FALSE
  ) +

  # 2. dummy layer that only feeds the legend (square keys)
  geom_point(
    data = tibble(name = names(region_col), x = NA_real_, y = NA_real_),
    aes(x, y, fill = name),
    shape        = 22,        # solid square
    size         = 6,
    stroke       = 0,
    show.legend  = TRUE,
    inherit.aes  = FALSE
  )+

  ## ---- scales -----------------------------------------------------------
  scale_fill_manual(values = region_col, name = "Region") +
  scale_colour_manual(values = region_col, guide = "none") +

  guides(fill = guide_legend(
    nrow       = 2,                 # 1 column ⇒ 4 rows
    byrow      = TRUE,
    keywidth   = unit(0.6, "cm"),   # square keys
    keyheight  = unit(0.6, "cm")
  )) +
  ## ---- labels & theme ---------------------------------------------------
  labs(
    x = NULL,
    y = "Trends of HW intensity (°C per year)"
  ) +
  theme_minimal(base_size = 18) +
  theme(
    plot.background   = element_rect(fill = "white", colour = NA),
    legend.text       = element_text(size = 13),

    legend.position       = c(0.02, 0.15),   # (x, y) in npc units
    legend.justification  = c(0, 1),         # anchor by its top-left corner
    legend.direction      = "vertical",      # keeps items vertical
    legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA, linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4),

    legend.key.width      = unit(2, "cm"),    # leave if you want wider keys
    axis.text.x = element_blank()
  )
  # ylim(c(0,0.23))

plot_boxplot

ggsave("Manuscript/Figs/trend_intensity_boxplot.png", plot_boxplot, dpi = 300, height =575*3, width = 1319*3, units = "px")

In [ ]:
library(magick)

# ── 1. read the four individual plots ───────────────────────────────────
img_a <- image_read("Manuscript/Figs/Map_MedSea_intensity.png")
img_b <- image_read("Manuscript/Figs/line_intensity.png")
img_c <- image_read("Manuscript/Figs/trend_intensity.png")
img_d <- image_read("Manuscript/Figs/trend_intensity_boxplot.png")

# ── 2. add a corner label to each plot ──────────────────────────────────
stamp <- function(img, letter,
                  size = 80,               # point size of the label
                  gravity = "northwest",   # top-left corner
                  offset  = "+20+15") {    # x,y offset from the corner
  image_annotate(
    img, letter,
    font      = "Helvetica-Bold",
    size      = size,
    gravity   = gravity,
    location  = offset,
    color     = "black",
    boxcolor  = "white",   # gives a small white rectangle behind the letter
    weight    = 700        # ensures the stroke is bold
  )
}

img_a <- stamp(img_a, "A")
img_b <- stamp(img_b, "B")
img_c <- stamp(img_c, "C")
img_d <- stamp(img_d, "D")

# ── 3. assemble the composite exactly as before ─────────────────────────
row1  <- image_append(c(img_a, img_b), stack = FALSE)  # side-by-side
row2  <- image_append(c(img_c, img_d), stack = FALSE)

combo <- image_append(c(row1, row2),   stack = TRUE)   # one above the other

image_write(combo, path = "Manuscript/Figs/plot_intensity.png")

### 3.3.3 Duration

In [ ]:
library(sf)
library(ggplot2)
library(rnaturalearth)
library(rnaturalearthdata)
library(tidyverse)
library(terra)
library(viridisLite)
library(ggspatial)
library(Utilities.Package)


region_col <- c(
  # greens
  "Levantine Sea" = "#008F1A",   # brighter pine-green
  "Sea of Sicily" = "#6B9C3C",   # vivid olive-sage
  
  # blues / teals / purples
  "Alboran Sea"   = "#414E7C",   # richer slate-blue
  "Gulf of Lyon"  = "#266A78",   # clearer Mediterranean teal
  "Adriatic sea"  = "#583C68",   # deeper indigo-violet
  
  # yellows / ochres
  "Balearic Sea"  = "#A98C3D",   # crisp mustard-ochre
  "Aegean Sea"    = "#D7A543",   # sun-baked golden-ochre
  
  # reds / oranges / browns
  "Tyrrhenian Sea"= "#A02A2A",   # vivid brick-red
  "Algerian Sea"  = "#DF5513",   # bright burnt-orange
  "Ionian Sea"    = "#705943"    # warmer earthy-taupe
)
# Load your shapefile (replace with your path)
med_regions <- st_read("Data/mask_Med_Sea/More_regions_Med_Sea.shp") %>% 
  dplyr::filter(!str_detect(name, "Black"))

# Get world land polygons
world <- ne_countries(scale = "large", returnclass = "sf")


filelist <- "Outputs/SST/nc/duration_mean" %>% 
  list.files(full.names = T) %>% 
  as_tibble() %>% 
  rename(path = value) %>% 
  mutate(file = gsub(".*/","",path),
         year = substr(file,15,18))


stk <- filelist$path %>% 
  rast() 

names(stk) <- filelist$year

# stk[is.na(stk)] <- 0  

stk_avg <- mean(stk, na.rm = T)

map_duration <- ggplot() +
  ## ── data layers ─────────────────────────────────────────────
  tidyterra::geom_spatraster(data = stk_avg,
                             aes(fill = mean)) + 
  geom_sf(data = world, fill = "grey80", color = NA) +
  ## ── colour scales ───────────────────────────────────────────
  scale_fill_gradientn(
    colours  = turbo(10),
    na.value = "transparent",
    name     = "Mean duration of HW events (days)"
  ) +
    ## ── horizontal guide with title on top ──────────────────────
  guides(
    fill = guide_colourbar(
      direction      = "horizontal",
      title.position = "top",
      label.position = "bottom",            # optional – labels below bar
      barwidth       = unit(5, "cm"),       # tweak length
      barheight      = unit(0.45, "cm")     # tweak thickness
    )
  ) +
  ## ── map extent ──────────────────────────────────────────────
  coord_sf(xlim = c(-10, 40), ylim = c(30, 47), expand = FALSE) +
  ## ── *** scale bar *** ───────────────────────────────────────
  annotation_scale(
    location   = "bl",   # bottom-left
    width_hint = 0.25,   # ≈25 % of the map width
    line_width = 0.6,
    text_cex   = 2,
    pad_x      = unit(0.4, "cm"),
    pad_y      = unit(0.4, "cm")
  ) +
  ## ── theme ───────────────────────────────────────────────────
  theme_minimal(base_size = 18) +
  theme(
    panel.grid        = element_line(color = "gray90"),
    panel.background  = element_rect(fill = "aliceblue"),
    plot.background   = element_rect(fill = "white", color = NA),

    # move legend INSIDE, upper-left; anchor by its top-left corner
    legend.position       = c(0.02, 0.98),   # npc coords (x = 2 %, y = 98 %)
    legend.justification  = c(0, 1),
    legend.direction      = "horizontal",

    legend.title.align    = 0.5,             # centre title
    legend.text           = element_text(size = 13),

    # give legend a white background & a bit of padding
    legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA,linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4)
  )

map_duration

ggsave("Manuscript/Figs/Map_MedSea_duration.png", map_duration, dpi = 300, height =575*3, width = 1319*3, units = "px")

############ Line plot by region
med_regions <- st_transform(med_regions, crs(stk))

# med_regions <- st_zm(med_regions)

zone_rast <- terra::rasterize(med_regions, stk[[1]], field = 1:nrow(med_regions))
plot(zone_rast)
# Function that terra::zonal will apply: returns a 3-element vector
qfun <- function(values, ...) {
  mean(values, na.rm = TRUE)
}

# zonal() summarises **all layers at once**
z_raw <- terra::zonal(stk, zone_rast, fun = qfun)

z_df <- as.data.frame(z_raw)  %>% 
  mutate(region = med_regions$name) %>% 
  dplyr::select(-layer) %>% # attach names
  pivot_longer(c(-region),
               names_to   = "year",
               values_to = "mean") %>% 
  mutate(year = as.numeric(year))


plot_lines <- ggplot(z_df, aes(x = year, y = mean, colour = region)) +
  geom_point(alpha = 0.2, show.legend = F) +
  geom_line(alpha = 0.2, show.legend = F) +
  scale_colour_manual(values = region_col) +

  geom_smooth(
    method    = "gam",
    linewidth = 1.2,
    alpha     = 0.05,
    formula     = (y) ~ s(x, k = 3),  # <-- k goes here
    show.legend = F) +

  ## ── 1. force a ONE-column legend ────────────────────────────
  guides(
    colour = guide_legend(ncol = 1)   # 1 column, items stacked
  ) +
  # ylim(c(0,11.25))+

  labs(colour = "Regions", x = "Year", y = "Mean duration of HW events (days)") +

  ## ── 2. theme tweaks to pin legend inside, top-left ──────────
  theme_minimal(base_size = 18) +
  theme(
    plot.background   = element_rect(fill = "white", colour = NA),
    legend.text       = element_text(size = 13),

    legend.position       = c(0.02, 0.98),   # (x, y) in npc units
    legend.justification  = c(0, 1),         # anchor by its top-left corner
    legend.direction      = "vertical",      # keeps items vertical
    legend.background     = element_rect(fill = "white", colour = NA, linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4),

    legend.key.width      = unit(2, "cm")    # leave if you want wider keys
  )

plot_lines

ggsave("Manuscript/Figs/line_duration.png", plot_lines, dpi = 300, height =575*3, width = 1319*3, units = "px")
###################### MAP of TREND

df_map_duration_long <- stk %>%
  as.data.frame(xy = T) %>% 
  mutate(ID = c(1:nrow(.))) %>% 
  pivot_longer(-c(ID,x,y), values_to = "value", names_to = "year") %>% 
  mutate(value = case_when(is.na(value) ~ 0,
                           T ~ value)) %>% 
  mutate(year = as.numeric(year)) %>% 
  group_by(ID) %>%
  summarise(trend = coef(lm(value ~ year))[2])

df_duration <- stk %>%
  as.data.frame(xy = T) %>%  
  mutate(ID = c(1:nrow(.))) %>% 
  dplyr::select(c(x,y,ID))


df_trend <- df_map_duration_long %>% 
  # dplyr::filter(trend < quantile(trend, probs = 0.95)) %>% 
  left_join(df_duration, by = "ID") %>% 
  dplyr::select(-ID) %>% 
  relocate(x,y,trend)

rast_trend <- stk$`1982`
values(rast_trend) <- NA

pts <- vect(df_trend, geom = c("x", "y"), crs = crs(rast_trend))

rast_trend <- terra::rasterize(
  pts,             # what to burn
  rast_trend,         # target raster geometry
  field = "trend", # column to transfer
  fun   = "mean"   # how to handle duplicate points (mean, max, sum, …)
)

# Function that terra::zonal will apply: returns a 3-element vector
qfun <- function(values, ...) {
  mean(values, na.rm = TRUE)
}


# zonal() summarises **all layers at once**
z_raw_trend <- terra::zonal(rast_trend, zone_rast, fun = qfun)

# 1.  Add centroid geometry and its coordinates to `med_regions`
med_regions <- med_regions %>%                                 # already an sf multipolygon
  mutate(centroid = st_centroid(geometry),                     # geometry column is usually called `geometry`
         lon      = st_coordinates(centroid)[, 1],             # X
         lat      = st_coordinates(centroid)[, 2])             # Y
#   └─> use st_point_on_surface(geometry) instead of st_centroid() if you
#       must ensure the point falls inside every (multi)polygon

# 2.  Keep only what you want to join
centroids_tbl <- med_regions %>% 
  st_drop_geometry() %>%                                       # drops *all* sf geometries
  select(name, lon, lat)  

z_df_trend <- as.data.frame(z_raw_trend)  %>% 
  mutate(region = med_regions$name) %>% 
  dplyr::select(-layer) %>%
  mutate(mean = round(mean,3)) %>% 
  left_join(centroids_tbl, by = c("region" = "name"))


map_trend <- df_map_duration_long %>% 
  left_join(df_duration, by = "ID") %>% 
  ggplot() +
    ## ── data layers ───────────────────────────────────────────
    geom_tile(aes(x = x, y = y, fill = trend)) +
    geom_sf(data = world, fill = "grey80", colour = NA) +

    ## ── colour scale ──────────────────────────────────────────
    tidyterra::scale_fill_grass_c(
      name = "Trends of HW duration (days per year)",
      use_grass_range = FALSE 
      # trans = "asn"
    ) +

    ## ── 1. horizontal colour bar, title on top ───────────────
    guides(
      fill = guide_colourbar(
        direction      = "horizontal",
        title.position = "top",
        label.position = "bottom",            # labels under the bar (optional)
        barwidth       = unit(10, "cm"),       # adjust length
        barheight      = unit(0.45, "cm")     # adjust thickness
      )
    ) +

    ## ── map extent ───────────────────────────────────────────
    coord_sf(xlim = c(-10, 40), ylim = c(30, 47), expand = FALSE) +

    ## ── region means & colours ───────────────────────────────
    geom_label(data = z_df_trend,
              aes(x = lon, y = lat, label = mean, colour = region),
              show.legend = FALSE, fill = rgb(1,1,1,0.7), size = 5) +
    scale_colour_manual(values = region_col, name = "") +

    ## ── scale bar ────────────────────────────────────────────
    annotation_scale(
      location   = "bl",
      width_hint = 0.25,
      line_width = 0.6,
      text_cex   = 2,
      pad_x      = unit(0.4, "cm"),
      pad_y      = unit(0.4, "cm")
    ) +

    ## ── theme ────────────────────────────────────────────────
    theme_minimal(base_size = 18) +
    theme(
      panel.grid        = element_line(colour = "gray90"),
      panel.background  = element_rect(fill = "aliceblue"),
      plot.background   = element_rect(fill = "white", colour = NA),

      # 2. park legend INSIDE, top-left
      legend.position       = c(0.02, 0.98),  # npc coords: 2 % from left, 98 % up
      legend.justification  = c(0, 1),        # anchor by its top-left corner
      legend.direction      = "horizontal",

      legend.text           = element_text(size = 13),
      legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA, linewidth = 0.2),
      legend.margin         = margin(4, 4, 4, 4),

      axis.title            = element_blank()
    )

map_trend

ggsave("Manuscript/Figs/trend_duration.png", map_trend, dpi = 300, height =575*3, width = 1319*3, units = "px")


### BOXPLOT

# ── libraries ───────────────────────────────────────────────────────────
library(terra)   # raster/terra objects
library(sf)      # modern vector data
library(ggplot2) # plotting
library(dplyr)   # pipe-friendly helpers (optional)

# ── 1. load data ────────────────────────────────────────────────────────
r   <- rast_trend         # single- or multi-band
reg <- med_regions           # polygons

# Make sure both layers share the same CRS
reg <- st_transform(reg, crs(r))

# ── 2. extract values per polygon ───────────────────────────────────────
# terra::extract() returns a data.frame: one row per cell per polygon
#   ID   = polygon index (generated automatically)
#   lyr1 = raster value (one column per raster layer)
#
vals <- terra::extract(r, vect(reg))            # vect() converts sf → SpatVector

# ── 3. tidy up: attach region names / attributes ───────────────────────
# Suppose your regions layer has a column called "NAME"
ID_name <- reg %>% 
      as_tibble() %>%  
      dplyr::mutate(ID = c(1:10),
                    name = as_factor(name))  %>%       # ensure matching ID
      dplyr::select(ID, name)

vals <- vals %>% 
  dplyr::left_join(ID_name,by = "ID") %>% 
  dplyr::filter(mean != 0)%>%                       # your summary data-frame
  mutate(
    name = fct_reorder(name, mean, .desc = TRUE)  # order by mean, descending
  )

# ── 4. plot ─────────────────────────────────────────────────────────────
plot_boxplot <- ggplot() +
  # 1. data layer (legend suppressed)
  geom_boxplot(
    data          = vals,
    aes(x = name, y = mean, fill = name),
    colour        = "black",
    linewidth     = 0.7,
    # size = 2,
    # alpha         = .60,
    outlier.shape = 21,
    outlier.fill  = NA,
    show.legend   = FALSE
  ) +

  # 2. dummy layer that only feeds the legend (square keys)
  geom_point(
    data = tibble(name = names(region_col), x = NA_real_, y = NA_real_),
    aes(x, y, fill = name),
    shape        = 22,        # solid square
    size         = 6,
    stroke       = 0,
    show.legend  = TRUE,
    inherit.aes  = FALSE
  )+

  ## ---- scales -----------------------------------------------------------
  scale_fill_manual(values = region_col, name = "Region") +
  scale_colour_manual(values = region_col, guide = "none") +

  guides(fill = guide_legend(
    nrow       = 2,                 # 1 column ⇒ 4 rows
    byrow      = TRUE,
    keywidth   = unit(0.6, "cm"),   # square keys
    keyheight  = unit(0.6, "cm")
  )) +
  ## ---- labels & theme ---------------------------------------------------
  labs(
    x = NULL,
    y = "Trends of HW duration (days per year)"
  ) +
  theme_minimal(base_size = 18) +
  theme(
    plot.background   = element_rect(fill = "white", colour = NA),
    legend.text       = element_text(size = 13),

    legend.position       = c(0.02, 0.15),   # (x, y) in npc units
    legend.justification  = c(0, 1),         # anchor by its top-left corner
    legend.direction      = "vertical",      # keeps items vertical
    legend.background     = element_rect(fill = rgb(1,1,1,0.5), colour = NA, linewidth = 0.2),
    legend.margin         = margin(4, 4, 4, 4),

    legend.key.width      = unit(2, "cm"),    # leave if you want wider keys
    axis.text.x = element_blank()
  )
  # ylim(c(0,0.23))

plot_boxplot

ggsave("Manuscript/Figs/trend_duration_boxplot.png", plot_boxplot, dpi = 300, height =575*3, width = 1319*3, units = "px")

In [ ]:
library(magick)

# ── 1. read the four individual plots ───────────────────────────────────
img_a <- image_read("Manuscript/Figs/Map_MedSea_duration.png")
img_b <- image_read("Manuscript/Figs/line_duration.png")
img_c <- image_read("Manuscript/Figs/trend_duration.png")
img_d <- image_read("Manuscript/Figs/trend_duration_boxplot.png")

# ── 2. add a corner label to each plot ──────────────────────────────────
stamp <- function(img, letter,
                  size = 80,               # point size of the label
                  gravity = "northwest",   # top-left corner
                  offset  = "+20+15") {    # x,y offset from the corner
  image_annotate(
    img, letter,
    font      = "Helvetica-Bold",
    size      = size,
    gravity   = gravity,
    location  = offset,
    color     = "black",
    boxcolor  = "white",   # gives a small white rectangle behind the letter
    weight    = 700        # ensures the stroke is bold
  )
}

img_a <- stamp(img_a, "A")
img_b <- stamp(img_b, "B")
img_c <- stamp(img_c, "C")
img_d <- stamp(img_d, "D")

# ── 3. assemble the composite exactly as before ─────────────────────────
row1  <- image_append(c(img_a, img_b), stack = FALSE)  # side-by-side
row2  <- image_append(c(img_c, img_d), stack = FALSE)

combo <- image_append(c(row1, row2),   stack = TRUE)   # one above the other

image_write(combo, path = "Manuscript/Figs/plot_duration.png")

Copernicus, 2024. [Copernicus open access hub](https://browser.dataspace.copernicus.eu/).

Embury, O., Merchant, C.J., Good, S.A., Rayner, N.A., Høyer, J.L., Atkinson, C., Block, T., Alerskans, E., Pearson, K.J., Worsfold, M., McCarroll, N., Donlon, C., 2024. Satellite-based time-series of sea-surface temperature since 1980 for climate applications. Scientific Data 11, 326. <https://doi.org/10.1038/s41597-024-03147-w>

Hobday, A.J., Alexander, L.V., Perkins, S.E., Smale, D.A., Straub, S.C., Oliver, E.C., Benthuysen, J.A., Burrows, M.T., Donat, M.G., Feng, M., others, 2016. A hierarchical approach to defining marine heatwaves. Progress in oceanography 141, 227–238.

Hobday, A.J., Oliver, E.C., Gupta, A.S., Benthuysen, J.A., Burrows, M.T., Donat, M.G., Holbrook, N.J., Moore, P.J., Thomsen, M.S., Wernberg, T., others, 2018. Categorizing and naming marine heatwaves. Oceanography 31, 162–173.

Pisano, A., Nardelli, B.B., Tronconi, C., Santoleri, R., 2016. The new mediterranean optimally interpolated pathfinder AVHRR SST dataset (1982–2012). Remote Sensing of Environment 176, 107–116. <https://doi.org/10.1016/j.rse.2016.01.019>

Schlegel, R.W., Smit, A.J., 2018. <span class="nocase">heatwaveR</span>: A central algorithm for the detection of heatwaves and cold-spells. Journal of Open Source Software 3, 821. <https://doi.org/10.21105/joss.00821>